# 3D Packalgorithmus
Autor: Dominik Lipfert, <br />
Matrikelnummer: 67323, <br />
Studiengang:Wirtschaftsingenierwesen Master <br />
<br />
basierend auf dem Packalgorithmus von Reiplinger (2021)

## Gliederung
0. Klassen des Packalgorithmus
 - Class: Article
 - Class: Container
 - Class: Solution (=Individuum)
 - Class: Genetic Algorithm
 - Class: Packing Algorithm
 - Class: Grid Search
1. Datenvorverarbeitung 
2. Restriktionen in Cython
 - Platzierungsbedingungen
 - Bedingung für Traglast einzelner Packstücke
3. Anwendung & Ergebnisausgabe

In [106]:
import collections
import numpy as np
import open3d as o3d
from IPython.core.display import display, HTML
import random

import pandas as pd
import itertools
import time
import math
import copy
import matplotlib.pyplot as plt
from functools import reduce
from operator import add
import csv
from datetime import datetime
from IPython.core.display import display, HTML

%load_ext Cython


The Cython extension is already loaded. To reload it, use:
  %reload_ext Cython


C:\Users\andre\AppData\Local\Temp\ipykernel_28812\1020292281.py:4: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML
C:\Users\andre\AppData\Local\Temp\ipykernel_28812\1020292281.py:17: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [107]:
display(HTML("<style>.container { width:100% !important; }</style>"))

## 0. Klassen des Packalgorithmus

### Class: Article

In [108]:
class Article:
    def __init__(self, width, height, length, x, y, z, is_stack, package_ID, package_sequence_nr, package_weight):
        self.width = width # Packstückbreite
        self.height = height # Packstückhöhe
        self.length = length # Packstücklänge
        self.x = x # Breitenkoordinate
        self.y = y # Höhenkoordinate
        self.z = z # Längenkoordinate
        self.is_stack = is_stack # 0 wenn sich keine Packstücke auf dem Packstück befinden, sonst 1
        self.package_ID = package_ID # eindeuitge ID des Packstücks
        self.package_sequence_nr = package_sequence_nr # Entspritcht Reihenfolge, nachdem Packstücke in Container gepackt werden
        self.package_weight = package_weight # Gewicht des Packstücks
        self.supported_articles = [(0,0)] # Tupel mit (Packstück-ID, anteiliges Gewicht) von Packstücken, umittelbar auf diese Packstück-Instanz

    def set_is_stack_to_true(self):
        self.is_stack = True

    # Überprüfe beim beladen des Packstücks mit einer zusätzliche Last, ob die maximale Traglast des Packstücks durch diese überschritten wird
    def check_if_article_has_overload(self, weight_of_article_to_stack):
        
        # Definiere den Faktor, der vorgierbt das Wievielfache seines Eigengewicht ein Packstück tragen darf
        overload_factor = 1.0

        # Initialisiere die Variable, die bestimmt ob das Packstück überladen ist
        article_has_overload = False

        supported_weights = [supported_article[1] for supported_article in self.supported_articles] # Erzeuge eine Liste, welche die Last jedes einzelnen Packstücks enthät, das das Packstück bereits trägt
        supported_weight_sum = reduce(add, supported_weights) # Berechne die gesamte Last, die das Packstück bereits trägt

        if ((supported_weight_sum + weight_of_article_to_stack) > (self.package_weight * overload_factor)): # Prüfe ob die gesamte Last die das Pakcstück bereits trägt + die zusätzliche Last durch das neue Packstück die maximale Traglast übersteigt
            article_has_overload = True # Setzte die Variable, die bestimmt ob das Packstück überladen ist auf True
        
        return article_has_overload # Liefere die Information, ob das Packstück durch die zusätzliche Last überladen wird zurück

    # Fügt Packstück ID und die damit verbundene Last der Liste alle Packstücke, die durch diese Packstück getragen werden hinzu
    def add_supported_article(self, supported_article_ID, supported_article_weight):
        self.supported_articles.append((supported_article_ID, supported_article_weight))


### Class: Container

In [109]:
box = collections.namedtuple('box', 'width height length')

class Container:
    
    def __init__(self, container_width, container_height, container_length, container_costs, max_weight, rest_packages):
        self.container = box(container_width, container_height, container_length) # Container Dimensionen
        self.costs = container_costs # Containerkosten
        self.max_weight = max_weight # Maximaltraglast des Containers
        self.carried_weight = 0 # Initialisiere Variable für das gepackte Gewicht des Containers
        self.article_locations = [] # Initialisiere Liste, in welche alle Packstücke gespeichert werden, die der Container enthält
        self.volume_used = 0 # Initialisiere Variable für das genutzte Volumen des Containers
        self.volume_usage_rate = 0 # Initialisiere Variable für das Containerauslastung in Prozent

        '''Variablen die der SKA für die Containerbefüllung benötigt'''
        self.rest_packages = rest_packages # Initialisiere Liste mit allen noch nicht gepackten Packstücken
        self.packages_packed = [] # Speichere platzierte Packstücke separat ab. Hierüber werden später die platzierten Packstücke aus den modifizierten Bestelldaten entfernt.
        self.package_sequence_nr = 1 # Initialisierung der Variable für die Packreihenfolge, mit der nachvollzogen werden kann, in welcher Reihenfolge die Packstücke in den Container gepackt werden
        self.call_from_stack_building = False # Initialisiere Variable um zu definieren, ob das die Add_One_Raw Funktion durch eine Stack Building Funktion aufgerufen wird

        self.factor_overfilling = 1.05 # Parameter, welcher die maximale Überschreitung von der Reihenlänge vorgibt. 1,05 entspricht einer Überschreitung von 5 % der Reihenlänge.

        self.articles_from_width_extension = [] # Initialisiere Liste, welche durch die width_extension mit Tupeln befüllt wird, im Falle von call_from_stack_building = False bleibt diese Liste leer
        self.articles_from_length_extension = [] # Initialisiere Liste, welche durch die length_extension mit Tupeln befüllt wird, im Falle von call_from_stack_building = False bleibt diese Liste leer

        # Initialisiere Startkoordinaten für die Platzierung von Packstücken in dem Container
        self.main_box_width = 0 
        self.main_box_height = 0
        self.main_box_length = 0

        # Initialisiere verfügbare Länge auf dem Boden des Containers (genannt Schichtlänge)
        # Die Schichtlänge wird nach dem Platzieren einer Hauptreihe um die Länge der Hauptreihe reduziert
        # Sie gibt damit die initiale Leerraumlänge vor dem Platzieren eine Hauptreihe vor
        self.main_layer_length = container_length # Initialisierung mit gesamter Containerlänge des Containertyps

        # Für das Wall-Buikding:
        # Anfangs steht der gesamte Container zur Bildung einer Reihe zur Verfügung
        self.main_raw_width = container_width # Breite des Leerraums
        self.main_raw_length = container_length # Länge des Leerraums

        # Für das Raw & Layer Building:
        self.articles_in_raw = [] # Liste in welche alle Artikel einer Raw/Layer gespeichert werden

        
        '''Ende Variablen die der SKA für die Containerbefüllung benötigt'''

    def add_article(self, width, height, length, x, y, z, is_stack, package_ID, package_sequence_nr, package_weight):
        self.article_locations.append(Article(width, height, length, x, y, z, is_stack, package_ID, package_sequence_nr, package_weight)) # Füge Packstück hinzu
        self.volume_used += (width * height * length)/1000000 # Erhöhe das genutze Volumen des Containers um das Volumen des hinzugefügten Packstücks
        self.carried_weight += package_weight # Erhöhe das Containergewicht um das Gewicht des hinzugefügten Packstücks
        

    def remove_article(self, article):
        self.article_locations.remove(article)
        self.volume_used -= (article.width * article.height * article.length)/1000000 # Verringere das genutze Volumen des Containers um das Volumen des entfernten Artikels
        self.carried_weight -= article.package_weight # Verringere das Containergewicht um das Gewicht des entfernten Packstücks
    
    def set_volume_usage(self, container_volume_usage):
        self.volume_used = container_volume_usage

    # Berechne Containerauslastung in Prozent
    def set_volume_usage_rate(self):
        self.volume_usage_rate = (self.volume_used/((self.container.width*self.container.height*self.container.length)/1000000))*100 

    def Check_Overload_Of_Artcicles (self, selected_orientation):
    
        # Initialisiere Variable, die bestimmt, ob die Platzierung des übergebenen Packstücks zulässig ist
        placement_allowed = True

        # Erzeuge eine Listen mit den Informationen welche Packstücke der Stapelgrundfläche mit welcher Fläche das zu stapelnde Packstück tragen. In Form von Tupeln: (Packstück ID, tragende Fläche)
        # 1. Funktionsaufruf; Übergebe: zu packendes Packstück; Packstück welches die Basis der Stapelfläche bildet + alle Packstücke durch die die Stapelgrundfläche der Breite nach erweitert wurde
        # 2. Funktionsaufruf; Übergebe: zu packendes Packstück; alle Packstücke durch die die Stapelgrundfläche der Länge nach erweitert wurde
        list_with_covered_areas = Calculate_Overlapping_Areas(selected_orientation, self.articles_from_width_extension) + Calculate_Overlapping_Areas(selected_orientation, self.articles_from_length_extension)
        
        # Initialisiere die Packstück ID des zu packenden Packstücks
        package_ID_article_to_stack = selected_orientation[7]
        # Initialisiere das Gewicht des zu packenden Packstücks
        weight_article_to_stack = selected_orientation[8]
        
        # Initialisiere eine Variable, welche die gesamte Fläche auf der das zu packende Packstück aufliegt speichert
        area_used_of_base = 0
        # Berechne die gesamte Fläche auf der das zu packende Packstück aufliegt
        for article_base in list_with_covered_areas:
            area_used_of_base += article_base[1]

        # Weise das Gewicht des zu stapelnden Packstücks anteilig auf die Packstücke der Stapelgrundfläche zu, auf denen es aufliegt & überprüfe ob dadurch eine Überladung zustande kommt
        for article_base in list_with_covered_areas:
            area_article_base = article_base[1] # Initialisiere die Größe der tragenden Fläche des Packstücks, welches beladen wird
            package_ID_article_base = article_base[0] # Initialisiere die Packstück ID des Packstücks, welches beladen wird
            covered_weight = 0 # Initialisiere das Gewicht, welches das Packstück, welches beladen wird, zu tragen hat
                    
            if(len(list_with_covered_areas) == 1): # Überprüfe, ob ein Packstück der Grundfläche das gesamte zu stapelnde Packstück trägt
                covered_weight = weight_article_to_stack # Weise das Gemsatgewicht des zu stapelnden Packstücks dem einen Packstück zu
            
            else: # Mehrere Packstücke targen das zu packende Packstück
                covered_weight = area_article_base/area_used_of_base * weight_article_to_stack # Weise das Gewicht des zu stapelnden Packstücks dem Packstück anteilig zu (tragende Fläche des Packstücks / Gesamte tragende Fläche * Gewicht)
            
            # Überprüfe für das zu beladende Packstück, ob es überladen ist, falls das zu stapelnde Packstück (anteilig) darauf gestapelt wird
            for article in self.article_locations:
                if (article.package_ID == package_ID_article_base and article.check_if_article_has_overload(covered_weight)):
                    placement_allowed = False # Falls es überladen ist, erlaube die Platzierung des zu stapelnden Packstücks nicht

        # Überprüfe abschließend, ob das zu stapelnde Packstück platziert werden darf & füge das (anteilig) getragene Gewicht des zu stapelnden Packstücks allen als Stapelfläche dienene tragenden Packstücken hinzu    
        if (placement_allowed == True):
            for article_base in list_with_covered_areas:
                package_ID_article_base = article_base[0] # Initialisiere die Packstück ID des Packstücks, welches beladen wird
                for article in self.article_locations:
                    if (article.package_ID == package_ID_article_base):
                        article.add_supported_article(package_ID_article_to_stack, covered_weight) # Füge das getragene Gewicht der Liste aller getragenene Gewichte des Packstücks hinzu      
        
        return placement_allowed

    '''Funktionen des Single Knapsack Algortihmus'''
    # Heuristik zur Befüllung eines Containers
    def pack_container (self):
    
        # Führe Wallbuilding durch
        raw_length_list = self.run_Wall_Building()
        
        # Führe Postoptmierung durch 
        self.run_Post_Optimization(raw_length_list)
    
        return self.packages_packed

    def run_Wall_Building (self):

        # Solange Restartikel bzw. Restpackstücke übrig sind, bilde Reihen
        while len(self.rest_packages) > 0:
            rest_packages_old = len(self.rest_packages) # Variable, welche der Anzahl an verbleibenden Packstücke entspricht. Diese wird als Abbruchkriterium verwendet.
            
            self.factor_overfilling = 1.05  # Parameter, welcher die maximale Überschreitung von der Reihenlänge vorgibt. 1,05 entspricht einer Überschreitung von 5 % der Reihenlänge.
            
            self.call_from_stack_building = False # Setzte Variable auf False, die bestimmt ob die Add_One_Raw Funktion durch eine Stack Building Funktion aufgerufen wird

            # Rufe Methode zur Bildung einer Reihe auf
            # Erhalte: Liste mit Packstücklängen
            raw_length_list = self.Add_One_Raw (self.main_raw_width, self.main_raw_length, self.main_box_width, self.main_box_length, self.main_box_height)
            
            # Terminiere Wall-Building, sofern keine weiteren Packstücke und damit keine weitere Wall erzeugt werden konnte
            # In diesem Falle müssen genauso viele Restpackstücke existieren wie vor dem Raw-Building
            if len(self.rest_packages) == rest_packages_old: 
                break
            
            # Rufe Methode zur Bildung eines Stapels auf
            # Durchlaufe die bereits platzierten Packstücke. Jedes platzierte Packstück wird als Stapel angesehen.
            for article_packed in self.article_locations:
                self.Stack_Building (article_packed)
            # Nun ist eine Wall komplettiert

            # Setze Koordinate für neue Haptreihe
            self.main_box_length = self.main_box_length + max(raw_length_list) # Neue Längenposition entspricht alter Längenposition zuzüglich der maximalen Länge der platzierten Reihe
            self.main_box_width = 0 # Breitenposition wird zurückgesetzt
            self.main_box_height = 0 # Höhenposition wird zurückgesetzt
            
            # Errechne den neuen Leerraum für die nächste Hauptreihe auf dem Containerboden
            self.main_raw_length = self.main_layer_length - max(raw_length_list) # Verbleibender Leerraum entspricht der Schichtlänge abzüglich der maximalen Länge der platzierten Reihe
            self.main_raw_width = self.container.width # Verfügbare Reihenbreite wird auf Containebreite zurückgesetzt


        return raw_length_list


    def run_Post_Optimization(self, raw_length_list):
        
        # Bestimme minimale Packstückdimension
        # Die minimale Dimension eines Leerraums muss größer oder gleich der minimalen Packstückdimension sein --> Einsparung Rechenzeit
        min_dimension = 1000
        for package in self.rest_packages:
            dimensions = [package[0][2],package[0][3], package[0][4]]
            if min(dimensions) < min_dimension:
                min_dimension = min(dimensions)
                
        # Erzeuge Empty Spaces
        empty_spaces = self.Create_Empty_Spaces(min_dimension)

        # Befülle die einzelnen Leerräume
        for empty_space in empty_spaces:
            
            # Erzeuge den betrachteten Leerraum
            empty_space_width = empty_space[0]
            empty_space_length = empty_space[1]
            empty_space_height = empty_space[2]
            # Setze Leerraumkoordinaten
            empty_space_x = empty_space[3]
            empty_space_z = empty_space[5]
            empty_space_y = empty_space[4]
            
            # Eine Überschreitung der Reihenlänge ist nun nicht mehr erlaubt
            self.factor_overfilling = 1.0 
            
            # Speichere die Packstücke aus der Schicht in einer Liste ab
            articles_in_raw = []
            
            # Erzeuge eine Schicht auf dem Boden des Containers
            raw_length_list = self.Add_Single_Layer (raw_length_list, empty_space_width, empty_space_length, empty_space_x, empty_space_y, empty_space_z)
            
            # Entferne Packstücke aus der einzelnen Schicht von den verbleibenden Packstücken
            # Im Gegensatz zu den Modulen Stack-Building und Add_One_Raw, entfernt Add_Single_Layer die platzierten Packstücke nicht selbstständig          
            self.remove_packed_articles_from_rest_packages()  
            
            # Wiederhole Stapelvorgang
            for article_packed in self.article_locations:
                self.Stack_Building (article_packed)
                
        # Führe finale Stapelung mit zugelassenen Überhängen durch
        for article_packed in self.article_locations:

            self.Modified_Stack_Building (article_packed)


    def remove_packed_articles_from_rest_packages(self):
        # Entferne die platzierten Packstücke aus den verbleibenden Packstücken
        for package_to_remove in self.articles_in_raw:
            self.rest_packages.remove(package_to_remove)



    def Add_One_Raw (self, main_raw_width, main_raw_length, main_box_width, main_box_length, main_box_height):
    
        # Initialisiere Liste für Packstücke in der Hauptreihe
        self.articles_in_raw = []
        
        # Speichere die Längen-Dimensionen der Packstücke einer Reihe in einer Liste ab. 
        # Das Packstück mit der größten Länge gibt die Verschiebung der Längen-Koordinate (box_length) nach Erzeugung der Reihe vor.
        raw_length_list = []
        
        # Beginne mit der Reihenbildung
        for article in self.rest_packages: # Durchlaufe alle Packstücke
                    
            selected_orientation = () # Initialisiere ein Tupel, in welches die Orientierung des potenziell zu platzierenden Packstücks gespeichert wird

            for orientation in article: # Durchlaufe die möglichen Orientierungen eines Packstücks (sortiert nach größtmöglicher Grundfläche)
                                    
                # Überprüfe ob das betrachtete Packstück in der betrachteten Orientierung platziert werden kann
                if (Check_Placement(article, self.articles_in_raw, self.container, self.max_weight, self.carried_weight, orientation, main_raw_width, main_raw_length, main_box_length, main_box_height, self.factor_overfilling) == True):
                    
                    # Speichere die Daten über die Orientierung zwischen, welche potenziell platziert werden kann
                    possible_orientation = (orientation[2], orientation[4], orientation[3], main_box_width, main_box_height, main_box_length, False, orientation[1], orientation[5])
                    # Prüfe ob das Raw Building im Rahmen einer Stapelung durchgeführt wird
                    if (self.call_from_stack_building == True):

                        # Rufe Methode zur Prüfung der Überschreitung der maximalen Traglast auf
                        # Übergebe: Orientierung, Packstücke der Stapelgrundfläche aus width_extension, Packstücke der Stapelgrundfläche aus length_extension, welche Gewichtzuweisung aller gepackten Packstücke enthält
                        # Erhalte: Information, ob Platzierung zulässig
                        placement_allowed = self.Check_Overload_Of_Artcicles(possible_orientation)
                        
                        if (placement_allowed == True): # Prüfe ob die Orientierung platziert werden darf
                            selected_orientation = possible_orientation # Wähle Orientierung als zu platzierende Orientierung aus
                            break # Überprüfe keine weiteren Orientierungen

                    else:
                        # Wähle die Orientierung des potenziell zu platzierenden Packstücks als zu platzierendes Packstück
                        selected_orientation = possible_orientation
                        break # Überprüfe keine weiteren Orientierungen


            # Prüfe ob es eine zulässige Orientierung gibt                
            if len(selected_orientation) != 0:

                # Füge dem Container das Packstück in der selektierten Orientierung hinzu
                self.add_article(selected_orientation[0], selected_orientation[1], selected_orientation[2], selected_orientation[3], selected_orientation[4], selected_orientation[5], selected_orientation[6], selected_orientation[7], self.package_sequence_nr, selected_orientation[8])
                self.package_sequence_nr += 1 # Erhöhe den Zähler für die Packstückreihenfolge um 1
                self.packages_packed.append(article) # Erweitere die platzierten Packstücke
                self.articles_in_raw.append(article) # Erweitere die in der Reihe platzierten Packstücke

                raw_length_list.append(selected_orientation[2]) # Speichere die Packstücklänge in dem dafür vorgesehenen Array
                main_box_width = main_box_width + selected_orientation[0] # Erhöhe die Breitenkoordinate
                main_raw_width = main_raw_width - selected_orientation[0] # Verringere die verfügbare Reihenbreite
                
                # Aktualisiere die erlaubte Reihenlänge, falls noch kein Packstück in der Reihe platziert wurde
                if len(self.articles_in_raw) == 1: 
                    main_raw_length = selected_orientation[2]
                
                # Für alle anderen Packstücke wird der entstehende Leerräume innerhalb der Reihe berechnet und befüllt. Hierbei werden Nebenreihen erzeugt.
                elif len(self.articles_in_raw) > 1:
                    if (main_raw_length - selected_orientation[2]) > 0: # Wenn freie Länge größer Null, dann rufe die Methode zur Befüllung des Leeraums auf
                        
                        side_raw_length = main_raw_length - selected_orientation[2] # Berechne die Länge des entstehenden Nebenleerraums
                        side_raw_width = selected_orientation[0] # Die Breite des entstehenden Nebenleeraums entspricht der Breite des platzierten Packstücks
                        side_box_length = main_box_length + selected_orientation[2] # Startposition für die Längenkoordinate vorgeben
                        side_box_width = main_box_width - selected_orientation[0] # Startposition für Breitenkoordinate vorgeben 
                        side_box_height = main_box_height # Startposition für Höhenkoordinate vorgeben
                        
                        # Leerraum wird mit Nebenreihen befüllt
                        raw_length_list = self.Add_Single_Layer (raw_length_list, side_raw_width, side_raw_length, side_box_width, side_box_length, side_box_height)
        
        # Entferne die platzierten Packstücke aus den verbleibenden Packstücken
        self.remove_packed_articles_from_rest_packages()          
                        
        return raw_length_list         


    def Add_Single_Layer (self, raw_length_list, side_raw_width, side_raw_length, side_box_width, side_box_length, side_box_height):
             
        initial_width = side_raw_width # Setze initiale Breitenkoordinate für Nebenleerraum
        initial_pos_width = side_box_width # Setze initiale Breite für Nebenleerraum
        
        available_length = side_raw_length # Setze initiale Nebenleerraumlänge. Diese wird fortlaufend um die Länge der Nebenreihen reduziert.
        
        layer_completed = False # Variable, die als Abbruchkriterium für das Layer-Building fungiert. Kann keine weitere Nebenreihe erzeugt werden, wird diese auf True gesetzt.
        
        while layer_completed == False:
            
            # Generiere eine Liste, in der die Längen der Packstücke der Nebenreihen gespeichert werden.
            # Diese ist notwendig, sofern ein Packstück die Nebenreihenlänge überschreitet.
            side_length = []
            
            # Durchlaufe die verfügbaren Packstücke
            for article in self.rest_packages:

                selected_orientation = () # Initialisiere ein Tupel, in welches die Orientierung des potenziell zu platzierenden Packstücks gespeichert wird

                for orientation in article: # Durchlaufe die Orientierungen eines Packstücks
                    
                    # Überprüfe ob das betrachtete Packstück in der betrachteten Orientierung platziert werden kann
                    if (Check_Placement(article, self.articles_in_raw, self.container, self.max_weight, self.carried_weight, orientation, side_raw_width, side_raw_length, side_box_length, side_box_height, self.factor_overfilling) == True):
        
                        # Speichere die Daten über die Orientierung zwischen, welche potenziell platziert werden kann
                        possible_orientation = (orientation[2], orientation[4], orientation[3], side_box_width, side_box_height, side_box_length, False, orientation[1], orientation[5])
                        
                        # Prüfe ob das Raw Building im Rahmen einer Stapelung durchgeführt wird
                        if (self.call_from_stack_building == True):
                            
                            # Rufe Methode zur Prüfung der Überschreitung der maximalen Traglast auf
                            # Übergebe: Orientierung, Packstücke der Stapelgrundfläche aus width_extension, Packstücke der Stapelgrundfläche aus length_extension, welche Gewichtzuweisung aller gepackten Packstücke enthält
                            # Erhalte: Information, ob Platzierung zulässig
                            placement_allowed = self.Check_Overload_Of_Artcicles(possible_orientation)
                            
                            if (placement_allowed == True): # Prüfe ob die Orientierung platziert werden darf
                                selected_orientation = possible_orientation # Wähle Orientierung als zu platzierende Orientierung aus
                                break # Überprüfe keine weiteren Orientierungen

                        else:
                            # Wähle die Orientierung des potenziell zu platzierenden Packstücks als zu platzierendes Packstück
                            selected_orientation = possible_orientation
                            break # Überprüfe keine weiteren Orientierungen


                # Prüfe ob es eine zulässige Orientierung gibt                
                if len(selected_orientation) != 0:
                    
                    if selected_orientation[2] > available_length: # Wenn Packstücke die Länge der Hauptreihe überschreitet, füge diese Länge der Liste hinzu
                        length_to_add = side_box_length + selected_orientation[2]
                        raw_length_list.append(length_to_add)

                    self.add_article(selected_orientation[0],selected_orientation[1], selected_orientation[2], selected_orientation[3], selected_orientation[4], selected_orientation[5], selected_orientation[6], selected_orientation[7], self.package_sequence_nr, selected_orientation[8])
                    self.package_sequence_nr += 1 # Erhöhe den Zähler für die Packstückreihenfolge um 1
                    self.articles_in_raw.append(article) # Erweitere die in dem Nebenleerraum platzierten Packstücke
                    self.packages_packed.append(article) # Erweitere die platzierten Packstücke
                    side_length.append(selected_orientation[2]) # Füge die Länge des Packstücks in die Liste der Hauptreihenlänge hinzu
                            
                    side_raw_width = side_raw_width - selected_orientation[0] # Aktualisiere die verfügbare Breite der Nebenreibe
                    side_box_width = side_box_width + selected_orientation[0] # Aktualisiere die Breitenkoordinate für nächstes Packstück
                    
                    # Platziere nächstes Packstück in der Nebenreihe
            
            if len(side_length) > 0:   # Reduziere die Länge der verfügbaren Schicht, falls eine Nebenreihe komplettiert wurde
                available_length = available_length - max(side_length)
            
                # Setze die Länge der neuen Reihe auf die verfügbare Schichtlänge zurück. Aktualisere die Längenkoordinate für neue Nebenreihen.
                side_raw_length = available_length 
                side_box_length = side_box_length + max(side_length)

                # Setze verfügbare Breite für neue Nebenreihen zurück und setze die Breitenkoordinate zurück
                side_raw_width = initial_width
                side_box_width = initial_pos_width
                
            else:  # Wurde keine Packstücklänge in der Liste gespeichert, konnte auch keine weitere Nebenreihe erzeugt werden. Das Layer-Building gilt als abgeschlossen.
                layer_completed = True 
                
                
        return raw_length_list



    def Stack_Building (self, article_packed):
            
        # Füge alle Indexes der Packstücke, die als Basis für einen Stapel fungieren einer Liste hinzu. Hierüber wird später die Variable isStack auf True gesetzt
        stack_indices = []

        # Initialisiere Liste, welche durch die width_extension mit Tupeln befüllt wird, die alle Packstücke um die die Stapelfläche erweitert wurde enthalten. Tupel: (Packstück ID, Packstück Breite, Packstück Länge, Packstück Breitenkoordinate, Packstück Längenkoordinate)
        # Zusätzlich wird dieser Liste ein Tupel des ersten Packstücks der Stapelfläche hinzugefügt
        self.articles_from_width_extension = []
        # Initialisiere Liste, welche durch die length_extension mit Tupeln befüllt wird, die alle Packstücke um die die Stapelfläche erweitert wurde enthalten. Tupel: (Packstück ID, Packstück Breite, Packstück Länge, Packstück Breitenkoordinate, Packstück Längenkoordinate)
        self.articles_from_length_extension = []

        # Initialisiere Variable um zu definieren, ob das die Add_One_Raw Funktion durch eine Stack Building Funktion aufgerufen wird
        self.call_from_stack_building = True

        # Packstücke auf einem Stapel werden nicht übersprungen
        # change_sequence = False
        
        # Keine Überschreitung der Reihenlänge auf einem Stapel zugelassen
        self.factor_overfilling = 1.0
        
        # Wenn ein Packstück noch nicht als Basis für einen Stapel fungierte (isStack = False), dann erzeuge die Stapelfläche
        if article_packed.is_stack == False:
            stack_indices.append(self.article_locations.index(article_packed)) # Füge den Packstückindex der entsprechenden Liste hinzu
            stack_width = article_packed.width # Stapelbreite/Leerraumbreite entspricht der Breite des Packstücks
            stack_height = article_packed.height # Stapelhöhe entspricht der Höhe des Packstücks
            stack_length = article_packed.length # Stapellänge/Leerraumlänge entspricht der Breite des Packstücks
            stack_box_width = article_packed.x # Breitenkoordindate entspricht der Breitenkoordinate des Packstücks
            stack_box_height = article_packed.y + article_packed.height  # Höhenkoordindate entspricht der Höhenkoordinate des Packstücks zuzuglüch der Packstückhöhe 
            stack_box_length = article_packed.z # Längenkoordindate entspricht der Längenkoordinate des Packstücks
            
            # Füge Informationen des ersten Packstücks der Stapelfläche der Liste hinzu. Tupel: (Packstück ID, Packstück Breite, Packstück Länge, Packstück Breitenkoordinate, Packstück Längenkoordinate)
            self.articles_from_width_extension.append((article_packed.package_ID, article_packed.width, article_packed.length, article_packed.x, article_packed.z))
            
            # Generiere größtmögliche Stapelbreite
            stack_width, stack_length, stack_box_height, stack_indices = self.width_extension (stack_indices, stack_width, stack_length, stack_box_width, stack_box_length, stack_box_height)
            # Generiere größtmögliche Stapellänge
            stack_length, stack_indices = self.length_extension (stack_indices, stack_width, stack_length, stack_box_width, stack_box_length, stack_box_height)
        
            '''Führe Stapelvorgang durch'''
            stack_layer_completed = False # Abbruchkriterium für das Stack-Building

            stack_layer_length = stack_length # Initiale Leerraumlänge. Diese wird mit dem Platzieren von Hauptreihen auf dem Stapel fortlaufend reduziert.
            initial_stack_width = stack_width # Initiale Leerraumbreite
            initial_stack_box_width = stack_box_width # Initiale Breitenkoordinate auf dem Stapel

            while stack_layer_completed == False:
                
                rest_packages_before_stacking = len(self.rest_packages) # Variable, welche der Anzahl an verbleibenden Packstücke entspricht.
                
                # Rufe Methode zur Bildung einer Reihe auf
                stack_raw_length_list = self.Add_One_Raw(stack_width, stack_length, stack_box_width, stack_box_length, stack_box_height)
                
                # Wenn keine Packstücke platziert wurden, unterbreche das Stack-Building
                if len(stack_raw_length_list) == 0: 
                    stack_layer_completed = True

                else:
                    stack_box_length = stack_box_length + max(stack_raw_length_list) # Neue Längenkoordinate entspricht alter Längenkoordindate zuzüglich der größten Packstücklänge der platzierten Reihe
                    stack_box_width = initial_stack_box_width # Breitenkoordiante wird auf die initiale Breitenkoordinate gesetzt
                    
                    stack_layer_length = stack_layer_length - max(stack_raw_length_list) # Aktualisiere verfügbare Leerraumlänge
                    stack_length = stack_layer_length # Neue initiale Reihenlänge entspricht der verfügbaren Leerraumlänge
                    stack_width = initial_stack_width # Verfügbare Reihenbreite wird zurückgesetzt
                    
                # Setze die Variable IsStack auf True für Packstücke, die als Basis fungierten. Damit werden keine weiteren Packstücken auf diesen platziert.
                if len(self.rest_packages) < rest_packages_before_stacking:
                    for article_indice in stack_indices: # Durchlaufe hierzu die Indizes der Packstücke, die als Basis für den Stapel fungieren
                        article_to_change = self.article_locations[article_indice] # Wähle das Packstück aus der Lösungskomponente
                        # Setze nun die Variable isSTack auf True
                        article_to_change.set_is_stack_to_true()
            
            '''Stapelung beendet'''
                
        return


    def width_extension (self, stack_indices, stack_width, stack_length, stack_box_width, stack_box_length, stack_box_height):
    
        # Generiere größt mögliche Stapelbreite
        extendible_in_width = True
        while extendible_in_width == True:
            stack_width_old = stack_width # Variable, die der Stapelbreite vor der Erweiterung entspricht
            for article_to_extend in self.article_locations:
                if ((article_to_extend.x >= stack_width + stack_box_width) and # Packstück muss größere Breitekoordinate als betrachtetes Packstück aufweisen
                    (article_to_extend.x <= stack_width + stack_box_width + 5) and # Packstück muss in der Breite nahezu an den Stapel angrenzen
                    (article_to_extend.z ==  stack_box_length) and # Packstück muss dieselbe Längenkoordinate aufweisen
                    (article_to_extend.length <= stack_length + 5) and # Packstück muss in etwa diesselbe Länge aufweisen
                    (article_to_extend.length >= stack_length - 5) and # Packstück muss in etwa diesselbe Länge aufweisen
                    (article_to_extend.y + article_to_extend.height <= stack_box_height + 5) and  # Packstück muss in etwa diesselbe Höhe aufweisen
                    (article_to_extend.y + article_to_extend.height >= stack_box_height - 5) and  # Packstück muss in etwa diesselbe Höhe aufweisen
                    (article_to_extend.is_stack == False)): # Packstücke darf ebenso nicht als Basis für einen Stapel fungiert haben

                        stack_width = stack_width + article_to_extend.width # Falls Bedingungen erfüllt sind, erweitere Stapelbreite
                        
                        # Falls das Packstück ein leicht größere Höhe aufweist, passe die Höhenkoordinate an
                        if (article_to_extend.y + article_to_extend.height > stack_box_height):
                            stack_box_height = article_to_extend.y + article_to_extend.height

                        # Falls das Packstück ein kleinere Länge aufweist, passe die verfügbare Stapellänge an
                        if stack_length < article_to_extend.length:
                            stack_length = article_to_extend.length
                        
                        # Speichere den Index des Packstücks in einer Liste ab. Damit kann die Variable isStack nach dem Stapelvorgang auf True gesetzt werden
                        stack_indices.append(self.article_locations.index(article_to_extend))

                        # Füge Informationen der Liste hinzu, die alle Tupel der Packstücke um die die Stapelfläche erweitert wurde enthalten. Tupel: (Packstück ID, Packstück Breite, Packstück Länge, Packstück Breitenkoordinate, Packstück Längenkoordinate)
                        self.articles_from_width_extension.append((article_to_extend.package_ID, article_to_extend.width, article_to_extend.length, article_to_extend.x, article_to_extend.z)) 

            # Terminiere, sofern die Stapelbreite nicht vergößert werden konnte
            if stack_width == stack_width_old: 
                    extendible_in_width = False
        
        return stack_width, stack_length, stack_box_height, stack_indices


    def length_extension (self, stack_indices, stack_width, stack_length, stack_box_width,stack_box_length,stack_box_height):        
    
        # Es muss eine Menge an Packstücken identifiziert werden, die über die gesamte Breite des Stapels angrenzt
        # Weiterhin muss diese Menge an Packstücke über eine identische Länge verfügen
        extendible_in_length = True
        while extendible_in_length == True:
            width_check = stack_box_width # Variable zur Überprüfung ob die Menge an Packstücken an die gesamte Breite des Stapels angrenzt
            length_check = 0 # Variable zur Überprüfung ob die Menge an Packstücken über diesselbe Länge verfügen
            width_extension = 0
            LDU = 0 # Length Determining Unit: Packstück, welches die Länge der Erweiterung vorgibt

            temporary_indices = [] # Liste zur Speicherung der temporären Packstück-Indexes

            for article_to_extend in self.article_locations:
                if ((article_to_extend.x == width_check) and # Es muss ein Packstück mit der selben Breitenkoordinate geben
                    (article_to_extend.z >= stack_box_length + stack_length) and # Das Packstück muss in etwa in der Länge an den Stapel angrenzen
                    (article_to_extend.z <= stack_box_length + stack_length + 5) and # Das Packstück muss in etwa in der Länge an den Stapel angrenzen
                    (article_to_extend.y + article_to_extend.height <= stack_box_height + 5) and # Packstück muss in etwa diesselbe Höhe aufweisen
                    (article_to_extend.y + article_to_extend.height >= stack_box_height - 5) and # Packstück muss in etwa diesselbe Höhe aufweisen
                    (article_to_extend.is_stack == False)): # Das Packstück darf noch keine Basis eines Stapels darstellen

                        # Das zuerst identifizierte Packstück gibt die Länge des Hilfsstapels vor
                        if LDU == 0:
                            length_check = article_to_extend.length
                            LDU = 1

                        # Weist das Packstück  ein passende Länge auf, wird der Hilftstapel erweitert
                        if ((article_to_extend.length <= length_check + 5) and
                            (article_to_extend.length >= length_check - 5)):

                            width_check = width_check + article_to_extend.width # Erweitere die Breitenkoordinate des Hilfsstapels, damit weitere Packstücke zur Erweiterung in der Breite identifziert werden können
                            width_extension = width_extension + article_to_extend.width # Erweitere die Breite des Hilfsstapels
                            temporary_indices.append(self.article_locations.index(article_to_extend)) # Füge den Index den temporären Stapelindizes hinzu
                            if article_to_extend.length > length_check: # Passe die Länge des Hilfsstapels an, sofern die Hilfsstapellänge durch das Packstück im Rahmen der Toleranz überschritten wird
                                length_check = article_to_extend.length
                            
                            # Füge Informationen der Liste hinzu, die alle Tupel der Packstücke um die die Stapelfläche erweitert wurde enthalten. Tupel: (Packstück ID, Packstück Breite, Packstück Länge, Packstück Breitenkoordinate, Packstück Längenkoordinate)
                            self.articles_from_length_extension.append((article_to_extend.package_ID, article_to_extend.width, article_to_extend.length, article_to_extend.x, article_to_extend.z)) 

            # Konnte eine Menge an Packstücke gefunden werden, welche über die gesamte Stapelbreite angrenzt, wird die endgültige Erweiterung der Stapelfläche durchgeführt
            # Der Stapel wird dann um die Länge des Hilfsstapels erweitert
            if ((width_extension >= stack_width - 5 )and 
                (width_extension <= stack_width + 5 )):
                    stack_length = stack_length + length_check
                    for indice in temporary_indices:
                        stack_indices.append(indice)     
            else: # Unterbreche, sofern die Breite des Hilfsstapels nicht der Breite des zu erweiternden Stapels entspricht
                break
            
            # Wurde kein passendes Packstück identiziert, unterbreche die Erweiterung
            if len(temporary_indices)==0:
                break
                        
        return stack_length, stack_indices



    def Create_Empty_Spaces(self, min_dimension):
    
        empty_space_pos_width = [] # Liste zur Speicherung der initialen Breitenposition aller Leerräume
        empty_space_pos_length = [] # Liste zur Speicherung der initialen Längenposition aller Leerräume
        empty_space_pos_height = 0 # Initiale Höhenposition der Leerräume = 0, da die Leerräume auf dem Boden des Containers gebildet werden
        empty_space_width = [] # Liste zur Speicherung der Länge aller Leerräume
        empty_space_length = [] # Liste zur Speicherung der Breite aller Leerräume
        empty_space_height = self.container.height # Höhe aller Leerräume entspricht der Containerhöhe
        empty_spaces = [] # Initialisierung Leerraumliste
        
        # Es wird eine Hilfsvariable definiert, welche der Längenposition einer Hauptreihe entspricht. Damit können Hauptreihen voneinander unterschieden werden.
        current_length = 0
        # Die erste Hauptreihe und damit auch der entsprechende reihenübergreifende Leeraum fangen bei der Längenposition 0 an
        # Diese wird der entsprechenden Liste hinzugefügt
        empty_space_pos_length.append(current_length)
        
        # Es wird eine Variable für die Anzahl der Haupreihen definiert
        number_of_walls = 1 
        
        # Bestimme Anzahl an Walls. Für jede Wall wird ein reihenübergreifender Leeraum berechnet.
        for article in self.article_locations:
            if ((article.y == 0)and # Höhenposition = 0: Wall muss einen Boden haben
                (article.x == 0)and # Breitenposition = 0: Wall muss einen Anfang haben
                (article.z != current_length)): # Längenposition muss sich von der Längenposition des vorherigen Packstücks unterscheiden
                number_of_walls = number_of_walls + 1 # Erhöhe Anzahl der Hauptreihen
                current_length = article.z # Aktualsiere die Längenposition bzw. die Hilfsvariable
                empty_space_pos_length.append(current_length) # Füge die Längenposition des Leerraums in die entsprechende Liste hinzu
        
        # Nun ist die Anzahl an Walls definiert. Weiterhin wurde für jeden reihenübergreifenden Leerraum dessen initiale Längenposition ermittelt.
        # Nun soll die initiale Breitenposition errechnet und der entsprechenden Liste hinzugefügt werden
        # Die initiale Breitenposition der reihenübergreifenden Leerräume entspricht dem Ende der Hauptreihen
        # Durchlaufe hierzu die Packstücke aus denselben Hauptreihen
        for i in range(number_of_walls):
            column_end = 0 # Hilfsvariable column_end wird zur Berechnung der initialen Breitenposition pro Leerraum verwendet. Initialisierung auf 0 für jede Hauptreihe.
            for article in self.article_locations:
                if ((article.y == 0)and # Höhenposition des Packstücks muss 0 sein
                    (article.z == empty_space_pos_length[i])and # Die Längenposition von Packstücken innerhalb einer Hauptreihe muss übereinstimmen
                    (article.x + article.width > column_end)): # Existiert ein Packstück, welches das Ende der Hauptreihe in der Breite überschreitet?
                        column_end = article.x + article.width # Falls ja, aktualisiere das Ende der Hauptreihenbreite
            empty_space_pos_width.append(column_end)
        
        # Jetzt sind die Initialkoordinaten für sämtliche reihenübergreifende Leerräume definiert
        # Als nächstes werden die Länge und Breite der Leerräume den entsprechenden Listen hinzugefügt
        # Durchlaufe hierzu erneut die Hauptreihen
        for i in range(number_of_walls):
            
            end = False # Hilfsvariable, welche angibt, ob das Ende eines Leerraums in der Länge erreicht ist
            
            # Verfügbare Leerraumbreite entspricht der Containerbreite abzüglich der initialen Breitenposition des Leerraums
            space_width = self.container.width - empty_space_pos_width[i]
            empty_space_width.append(space_width) # Füge die Breite des Leerraums in die entsprechende Liste hinzu
            
            # Bestimmen nun die maximale Leerraumlänge
            # Durchlaufe hierzu die bereits platzierten Packstücke und schaue, ob ein Packstück den Leerraum in der Länge durchkreuzt
            for article in self.article_locations:
                if ((article.z > empty_space_pos_length[i]) and # Ein solches Packstück muss über eine größere initiale Längenposition als der Leerraum verfügen
                    (article.y == 0 ) and #Ein solches Packstück muss sich auf dem Boden des Containers befinden
                    (article.y + article.width > empty_space_pos_width[i])): # Ein solches Packstück muss den Leerraum in der Breite schneiden. Breitenpositon + Breite des Packstücks > Breitenposition des Leerraums
                        space_length = article.z - empty_space_pos_length[i] # Falls ein Packstück durchkreuzt, errechnet sich die Leerraumlänge aus der Längenposition des identifzierten Packstücks abzüglich der intialen
                                                                            # Längenposition des Leerraums
                        empty_space_length.append(space_length) # Füge die Leerraumlänge der entsprechenden Liste hinzu
                        end = True # Ende des Leeraums ist erreicht
                        break
            
            # Wurde kein Packstück identifiziert, welches den Leerraum in der Länge durchkreuzt, so reicht die Leerrlänge bis zur Containerwand
            if end == False:
                space_length = self.container.length - empty_space_pos_length[i] # Leerraumlänge = Containerlänge - initiale Längenposition des Leerraums
                empty_space_length.append(space_length) # Füge die Leerraumlänge der entsprechenden List hinzu
        
        # Füge die Leerräume mit den enstprechenden Informationen (Dimensionen, Koordianten) der Leerraumliste hinzu
        for i in range(len(empty_space_pos_width)):
                if ((empty_space_width[i] > min_dimension) and # Länge und Breite des Leerraums müssen größer sein als die minimale Packstückdimension
                    (empty_space_length[i] > min_dimension)): 
                        empty_spaces.append((empty_space_width[i], empty_space_length[i], empty_space_height ,empty_space_pos_width[i], empty_space_pos_length[i], empty_space_pos_height))
        
        # Überprüfe ob es Schnittpunkte gibt
        # Führe hierzu paarweise Vergleiche zwischen den Leerräumen durch
        
        abbruch = False # Hilfsvariable, die als Abbruchkriterium fungiert
        while abbruch == False:
            
            length_empty_spaces_old = len(empty_spaces) # Hilfsvariable, welche der Anzahl an Leerräumen entspricht. Damit wird das Abbruchkriterium beeinflusst. 
            # Durchlaufe die Leerräume
            for i in range(len(empty_spaces)-1):
                # Prüfe zunächst, ob der betrachtete Leerraum in der Länge den darauffolgenden Leerraum durchkreuzt
                if (empty_spaces[i][1] + empty_spaces[i][4] > empty_spaces[i+1][4]): # Es musss gelte: Längenposition des Leerraums + Länge des Leerraums > Längenposition des nächsten Leerraums
                    # Falls Leerraum in der Längen durchkreuzt, prüfe welcher Leerraum die größere Grundfläche und damit das größere Volumen aufweist
                    # Grundfläche errechnet durch Leerraumbreite * Leerraumlänge
                
                    # Entferne den nachfolgenden Leerraum, falls dieser eine kleinere Grundfläche als der betrachtete Leerraum aufweist
                    if empty_spaces[i][0]*empty_spaces[i][1] > empty_spaces[i+1][0]*empty_spaces[i+1][1]: 
                        empty_spaces.remove(empty_spaces[i+1]) 
                        break
                        
                    # Bei Gleichstand: Selektiere den Leerraum mit der größeren Länge
                    elif empty_spaces[i][0]*empty_spaces[i][1] == empty_spaces[i+1][0]*empty_spaces[i+1][1]:
                        if empty_spaces[i][1] > empty_spaces[i+1][1]:
                            empty_spaces.remove(empty_spaces[i+1])
                            break
                        else:
                            empty_spaces.remove(empty_spaces[i])
                            break
                            
                    # Entferne den betrachteten Leerraum, falls dieser eine kleinere Grundfläche als der nachfolgende Leerraum aufweist
                    else:
                        empty_spaces.remove(empty_spaces[i])
                        break
                        
            # Unterbreche, falls keine weiteren Überschneidungen existieren
            if len(empty_spaces) == length_empty_spaces_old:
                abbruch = True
                                    
        return empty_spaces # Gebe sämtliche Leerräume zurück


    def Modified_Stack_Building (self, article_packed):
        global packplan
        packplan = []
        stack_indices = [] # Liste zur Speicherung der Indizes der Packstücke, die als Basis für den Stapel fungieren

        # Initialisiere Liste, welche durch die width_extension mit Tupeln befüllt wird, die alle Packstücke um die die Stapelfläche erweitert wurde enthalten. Tupel: (Packstück ID, Packstück Breite, Packstück Länge, Packstück Breitenkoordinate, Packstück Längenkoordinate)
        self.articles_from_width_extension = []
        # Initialisiere Liste, welche durch die length_extension mit Tupeln befüllt wird, die alle Packstücke um die die Stapelfläche erweitert wurde enthalten. Tupel: (Packstück ID, Packstück Breite, Packstück Länge, Packstück Breitenkoordinate, Packstück Längenkoordinate)
        self.articles_from_length_extension = []

        if article_packed.is_stack == False:

            stack_indices.append(self.article_locations.index(article_packed))
            stack_width = article_packed.width # Stapelbreite entspricht der Breite des Packstücks
            stack_height = article_packed.height # Stapelhöhe entspricht der Höhe des Packstücks
            stack_length = article_packed.length # Stapellänge entspricht der Breite des Packstücks
            stack_box_width = article_packed.x # Breitenkoordindate entspricht der Breitenkoordinate des Packstücks
            stack_box_height = article_packed.y + article_packed.height  # Höhenkoordindate entspricht entspricht der Höhenkoordinate des Packstücks zuzuglüch der Packstückhöhe 
            stack_box_length = article_packed.z # Längenkoordindate entspricht der Längenkoordinate des Packstücks

            # Füge Informationen des ersten Packstücks der Stapelfläche der Liste hinzu. Tupel: (Packstück ID, Packstück Breite, Packstück Länge, Packstück Breitenkoordinate, Packstück Längenkoordinate)
            self.articles_from_width_extension.append((article_packed.package_ID, article_packed.width, article_packed.length, article_packed.x, article_packed.z))

            # Generiere größtmögliche Stapelbreite
            stack_width, stack_length, stack_box_height, stack_indices = self.width_extension (stack_indices, stack_width, stack_length, stack_box_width, stack_box_length, stack_box_height)
            # Generiere größtmögliche Stapellänge
            stack_length, stack_indices = self.length_extension (stack_indices, stack_width, stack_length, stack_box_width, stack_box_length, stack_box_height)

            # Führe Stapelvorgang durch. Jetzt jedoch mit zugelassenen Überhängen.
            stack_completed = False
            while stack_completed == False:

                stack_raws = []  # Liste zur Speicherung der platzierten Packstücke
                stack_raw_length_list = []  # Liste zur Speicherung der Längen platzierter Packstücke
    
                stack_raw_width = stack_width  # Initiale Stapelbreite bzw. Breite des Leerraums
                
                # Durchlaufe die verbleibenden Packstücke
                for article_to_pack in self.rest_packages:                  
                    selected_orientation = () # Initialisiere ein Tupel, in welches die Orientierung des potenziell zu platzierenden Packstücks gespeichert wird
                    for orientation in article_to_pack: # Durchlaufe die möglichen Orientierungen eines Packstücks (sortiert nach größtmöglicher Grundfläche)

                        # Prüfe ob das Packstück auf den Stapel passt & keine anderen Packstücke überschneidet
                        if (Check_Placement_Modified_Stacking (self.container, self.max_weight, self.carried_weight, self.article_locations, orientation, stack_raw_width, stack_length, stack_box_length, stack_box_height, stack_box_width) == True):

                            # Falls  Bedingungen eingehalten werden, füge Orientierung der Liste hizu
                            possible_orientation = (orientation[2], orientation[4], orientation[3], stack_box_width, stack_box_height , stack_box_length, False, orientation[1], orientation[5])
                            
                            # Rufe Methode zur Prüfung der Überschreitung der maximalen Traglast auf
                            # Übergebe: Orientierung, Packstücke der Stapelgrundfläche aus width_extension, Packstücke der Stapelgrundfläche aus length_extension, welche Gewichtzuweisung aller gepackten Packstücke enthält
                            # Erhalte: Information, ob Platzierung zulässig
                            placement_allowed = self.Check_Overload_Of_Artcicles(possible_orientation)

                            if (placement_allowed == True): # Prüfe ob die Orientierung platziert werden darf
                                selected_orientation = possible_orientation # Wähle Orientierung als zu platzierende Orientierung aus
                                break # Überprüfe keine weiteren Orientierungen

                    # Prüfe ob es eine zulässige Orientierung gibt
                    if len(selected_orientation) != 0:

                        # Erweitere die Lösungskomponente
                        self.add_article(selected_orientation[0],selected_orientation[1], selected_orientation[2], selected_orientation[3], selected_orientation[4], selected_orientation[5], 
                                                    selected_orientation[6], selected_orientation[7], self.package_sequence_nr, selected_orientation[8])
                        self.package_sequence_nr += 1
                        self.packages_packed.append(article_to_pack) # Erweitere die gepackten Packstücke
                        stack_raws.append(article_to_pack) # Erweitere die Packstückliste des Stapels 

                        stack_raw_length_list.append(selected_orientation[2])  # Füge die Packstücklänge in die entsprechende Liste hinzu
                        stack_raw_width = stack_raw_width - selected_orientation[0] # Reduziere die verfügbare Reihenbreite
                        stack_box_width = stack_box_width + selected_orientation[0] # Akualsiere die Breitenkoordinate
                
                # Entferne die auf dem Stapel platzierten Packstücke von den verbleibenden Packstücken
                for article_to_remove in stack_raws:
                    self.rest_packages.remove(article_to_remove)

                # Setze die Variable IsStack auf True für Packstücke, die als Basis fungierten. Damit werden keine weiteren Packstücken auf diesen platziert.
                if len(stack_raws) > 0:
                    for article_indice in stack_indices:
                        article_to_change = self.article_locations[article_indice]
                        article_to_change.set_is_stack_to_true()
                
                # Falls keine Packstücke auf dem Stapel platziert wurden, unterbreche das Modified Stack-Building
                if len(stack_raws) == 0: 
                    stack_completed = True
                else: # Falls doch, passe die Längenkoordinate, die Breitenkoordinate und die Leerraumlänge an
                    stack_box_length = stack_box_length + max(stack_raw_length_list)
                    stack_box_width = article_packed.x
                    stack_length = stack_length - max(stack_raw_length_list)

        for artloc in self.article_locations:
            packplan_package = [artloc.width, artloc.height, artloc.length, artloc.x, artloc.y, artloc.z, artloc.is_stack, artloc.package_ID, artloc.package_sequence_nr, artloc.package_weight]
            packplan.append(packplan_package)
   
    ''' Ende Funktionen des Single Knapsack Algortihmus'''



    '''Funktionen zur Visualisierung'''
    def get_lineset(self, offset_x=0, offset_y=0, offset_z=0):
        points = [[0+offset_x, 0+offset_y, 0+offset_z],
                  [self.container.width+offset_x, 0+offset_y, 0+offset_z],
                  [0+offset_x, self.container.height+offset_y, 0+offset_z],
                  [self.container.width+offset_x, self.container.height+offset_y, 0+offset_z],
                  [0+offset_x, 0+offset_y, self.container.length+offset_z],
                  [self.container.width+offset_x, 0+offset_y, self.container.length+offset_z],
                  [0+offset_x, self.container.height+offset_y, self.container.length+offset_z],
                  [self.container.width+offset_x, self.container.height+offset_y, self.container.length+offset_z]]

        lines = [[0, 1], [0, 2], [1, 3], [2, 3], [4, 5], [4, 6], [5, 7], [6, 7], [0, 4], [1, 5], [2, 6], [3, 7]]
        colors = [[0, 0, 0] for i in range(len(lines))]

        line_set = o3d.geometry.LineSet(points=o3d.utility.Vector3dVector(points), lines=o3d.utility.Vector2iVector(lines))
        line_set.colors = o3d.utility.Vector3dVector(colors)    
        return line_set

    def save_packplan(self):
        print("Packplan")
        self.timestamp = datetime.now().strftime("%Y-%m-%d %Hh%Mm%Ss")
        file_path_packplan = 'Simulationen//' + str(self.timestamp) + ' BI01' + ' Simulation Packplan.csv'
        pckpln = []
        for artloc in self.article_locations:
            print(artloc.width, artloc.height, artloc.length, artloc.x, artloc.y, artloc.z, artloc.package_weight)
            package = [artloc.width, artloc.height, artloc.length, artloc.x, artloc.y, artloc.z, artloc.is_stack, artloc.package_ID, artloc.package_sequence_nr, artloc.package_weight]
            pckpln.append(package)
        results_packplan = pd.DataFrame(pckpln)
        pckpln = []
        results_packplan.to_csv(path_or_buf=file_path_packplan, header=False, index=False)

    def save_container(self):
        self.timestamp = datetime.now().strftime("%Y-%m-%d %Hh%Mm%Ss")
        file_path_container = 'Simulationen//' + str(self.timestamp) + ' BI01' + ' Simulation Container.csv'
        container = []
        print(self.container[0], self.container[1], self.container[2])
        cntnr = [self.container[0], self.container[1], self.container[2]]
        container.append(cntnr)
        results_container = pd.DataFrame(container)
        container = []
        results_container.to_csv(path_or_buf=file_path_container, header=False, index=False)


    
    def get_geometry(self, offset_x=0, offset_y=0, offset_z=0):
        line_set=self.get_lineset(offset_x, offset_y, offset_z)
        
        geo_list=[line_set]
        for artloc in self.article_locations:
            mesh_box = o3d.geometry.TriangleMesh.create_box(width=artloc.width, height=artloc.height, depth=artloc.length)
            mesh_box.translate(np.array([artloc.x+offset_x,artloc.y+offset_y,artloc.z+offset_z]))   
            mesh_box.compute_vertex_normals()

            rvalue=random.random()
            mesh_box.paint_uniform_color([rvalue, 1-rvalue, random.random()])    
            geo_list.append(mesh_box)
        return geo_list
    
    def __repr__(self):
        return str(self.container) + "\n    " + "\n    ".join(str(x) for x in self.article_locations)
    '''Ende Funktionen zur Visualisierung'''

### Class: Solution (=Individual)

In [110]:
class Solution:
    
    def __init__(self, modified_order_data, container_data):
        self.containers=[] # Liste der Container, aus der die Lösung besteht
        self.container_number = 0 # Anzahl der Container, aus der die Lösung besteht
        self.usage_average = 0 # durschnittliche Containerauslastung der Lösung
        self.costs = 0 # Summe der Kosten aller Container in der Lösung

        # Variablen des Greedy Verfahrens
        self.modified_order_data = modified_order_data
        self.container_data = container_data

        self.packages_packed_list = [] # Liste in der die platzierten Packstücke aller Containertypen abgespeichert werden
        self.container_to_add_list = [] # Liste in der die einzelnen Lösungskomponenten bzw. Instanzen abgespeichert werden
        self.usage_list = [] # Liste in der die Auslastungen aller Containertypen abgespeichert werden

        self.index_selected_container = 0 # Index des gewählten Containers
        self.usage_selected = 0 # Volumenauslastung des gewählten Containers


    # Füge einen übergebenen Container der Lösung hinzu
    def add_container(self, container):
        self.containers.append(container) # Füge Container der Lösung hinzu
        self.costs += container.costs # Erhöhe die Kosten der Lösung
        self.container_number += 1 # Erhöhe die Anzahl der Container in der Lösung

    # Berechne die durschnittliche Volumenauslastung der Lösung
    def set_usage_average(self):
        usage_total = 0 # Initialisieren Volumenauslastung

        for container in self.containers:
            usage_total += container.volume_usage_rate # Summiere Volumenauslastung aller Container in der Lösung auf
        
        self.usage_average = usage_total/len(self.containers) # Teile Summe durch Containeranzahl

    '''Funktionen für das Greedy Verfahren'''
    def calculate_solution (self):
        # Packe einen neuen Container, solange verbleibende Packstücke existieren
        while len(self.modified_order_data) > 0:

            
            # Entferne die Daten aus den Listen
            self.packages_packed_list = []
            self.container_to_add_list = []
            self.usage_list = []

            # Wende den SKA auf jeden Container-Typ an
            global rest_packages
            rest_packages = self.pack_each_container_type()

            # Nun liegen die Ergebnisse für jeden Containertyp vor
            # Das Greedy-Verfahren selektiert nun den Containertyp mit der höchsten Volumenauslastung
            self.index_selected_container = self.usage_list.index(max(self.usage_list)) # Index des Containertyps mit der höchsten Volumenauslastung, bei mehreren gleichen Werten wählt max() in Python den Index des ersten Wertes
            self.usage_selected = self.usage_list[self.index_selected_container] # Auslastung des Containertyps mit der höchsten Volumenauslastung

            
            # Prüfe, ob mehrere Container mit der selben Auslastung existieren, falls ja: wähle den größeren
            self.check_for_same_volume_usage_and_select_bigger()
                             
            # Prüfe, ob es ein Container existiert, welcher alle verbleibenden Packstücke packt und wähle diesen falls die in der Funktion beschriebenen Voraussetzungen erfüllt sind
            self.last_container_optimization(rest_packages)
                
            # Selektiere nun die Lösungskomponente, die gepackten Packstücke, die Containerauslastung und das gepackte Gewicht
            packages_packed_selected = self.packages_packed_list [self.index_selected_container]
            container_to_add_selected = self.container_to_add_list [self.index_selected_container]
            self.usage_selected = self.usage_list [self.index_selected_container]
            
            # Füge die Lösungskomponente der Lösung hinzu
            self.add_container(container_to_add_selected)

            # Die Packstücke welcher in der selektierten Instanz gepackt wurden, müssen aus modifizierten Bestelldaten entfernt werden
            for package in packages_packed_selected:
                self.modified_order_data.remove(package)
        
        # Errechne die durchschnittliche Containerauslastung der Lösung
        self.set_usage_average()



    # Prüfe, ob mehrere Container mit der selben Auslastung existieren, falls ja: wähle den größeren
    def check_for_same_volume_usage_and_select_bigger(self):
        # Durchlaufe dazu die Liste mit den gespeicherten Containerauslastungen
        for index in range(len(self.usage_list)):
            if self.usage_list[index] == (self.usage_selected): # Überprüfe für jeden Container, ob er die selbe Auslastung hat, wie der Container mit der höchsten Auslastung
                if (self.container_to_add_list[index].container.width * self.container_to_add_list[index].container.height * self.container_to_add_list[index].container.length) > \
                    (self.container_to_add_list[self.index_selected_container].container.width * self.container_to_add_list[self.index_selected_container].container.height * self.container_to_add_list[self.index_selected_container].container.length): # Überprüfe bei gleicher Auslastung, ob der bisher selektierte Container kleiner ist
                    self.index_selected_container = index # Ändere den Index des selektierten Containers, falls der bisher selektierte kleiner war
                    self.usage_selected = self.usage_list[self.index_selected_container] # Ändere die Auslastung des selektierten Containers


    # Wende den SKA auf jeden Container-Typ an
    def pack_each_container_type(self):
        global rest_packages
        for container_type in self.container_data:
            global rest_packages

            # Sicherstellung, dass jedem Containertyp diesselben Packstücke übergeben werden
            rest_packages = [package for package in self.modified_order_data]
            
            # Anwendung SKA zur Generierung einer Lösungskomponente
            container_to_add = Container(container_type[0], container_type[2], container_type[1], container_type[3], container_type[4], rest_packages)
            packages_packed = container_to_add.pack_container()

            container_to_add.set_volume_usage_rate()# Berechne Containerauslastungsrate der Lösungskomponente

            # Füge die Ergebnisse für den betrachteten Containertyp den erzeugten Listen hinzu
            # Aber nur, falls mindestens ein Packstück platziert wurde
            if container_to_add.volume_used != 0:
                
                self.packages_packed_list.append(packages_packed) # Erweitere die Liste in der die platzierten Packstücke aller Containertypen abgespeichert werden
                self.container_to_add_list.append(container_to_add) # Erweitere die Liste in der die einzelnen Lösungskomponenten bzw. Instanzen abgespeichert werden
                self.usage_list.append(container_to_add.volume_usage_rate) # Erweitere die Liste in der die Auslastungen aller Containertypen abgespeichert werden

        return rest_packages
    
    # Prüft, ob es ein Container existiert, welcher alle verbleibenden Packstücke packt und wählt diesen falls die in der Funktion beschriebenen Voraussetzungen erfüllt sind
    def last_container_optimization(self, rest_packages):
            
            # Prüfe zunächst, ob Container existieren, welche alle restlichen Packstücke packen & ermittle daraus den Container mit der höchsten Auslastung
            container_rest_packages_volume_usage_rate = 0 # Initialisiere die Auslastung des Container, der alle restlichen Packstücke packt
            for index, container_packages in enumerate(self.packages_packed_list):
                if (len(container_packages) == len(self.modified_order_data)) and (self.container_to_add_list[self.index_selected_container].volume_usage_rate > container_rest_packages_volume_usage_rate):
                    container_rest_packages_volume_usage_rate = self.container_to_add_list[index].volume_usage_rate # Auslastung des Containers der alle restlichen packstücke packt & die höchste Auslastung hat
                    index_container_all_rest_packages = index # Index des Containers der alle restlichen packstücke packt & die höchste Auslastung hat

                    # Es gibt einen Container, der alle restlichen Packstücke packt

                    # Berechne das Volumen der nicht durch den aktuell selektierten Container gepackten Packstücke
                    remaining_articles = [article for article in rest_packages if article not in self.packages_packed_list [self.index_selected_container]] # Erzeuge eine Liste, die alle Packstücke enthält die durch den selektierten Container nicht gepackt wurden
                    volume_of_remaining_articles = 0 # Initialisiere das Gesamtvolumen der nicht gepackten Packstücke
                    for article in remaining_articles:
                        volume_of_remaining_articles += article[0][6] # Summiere die Volumen der Packstücke

                
                    # Nun wird ermittelt, ob der Container der alle restlichen Packstücke anstatt dem bisher selektierten Container verwendet werden soll
                    # Prüfe, ob es einen Containertypen gibt, der alle restlichen Packstücke tragen kann & dessen Ausalstung verrechnet mit dem aktuell selektierten Container im Mittel höher ist, als die Auslastung des Containers, der alle restlichen Packstücke packen kann
                    use_container_all_rest_packages = True # Initialisiere die Verwendung des Containers der alle restlichen Packstücke packt mit True
                    for container in self.container_data:
                        container_volume = container[0]*container[1]*container[2]/1000

                        # Prüfe, ob der Conrtainertyp ein größeres Volumen als alle restlichen Packstücke hat & ob die durchschnittliche Auslastung aus dem bisher selektierten Container und dem nächsten Container (befüllt mit allen Restartikeln) größer ist, als die Auslastung des Containers, der alle restlichen Packstücke packt
                        if volume_of_remaining_articles/container_volume < 1 and \
                            (self.container_to_add_list[self.index_selected_container].volume_usage_rate + volume_of_remaining_articles/container_volume) / 2 > self.container_to_add_list[index_container_all_rest_packages].volume_usage_rate:
                            # Trifft die beiden Bedingungen zu soll der Container der alle restlichen Packstücke tragen kann nicht verwendet werden
                            use_container_all_rest_packages = False

                    if (use_container_all_rest_packages): # Prüfe, ob der Container der alle restlichen Packstücke tragen kann verwendet werden soll
                        self.index_selected_container = index_container_all_rest_packages # Verwende den Container, der alle restlichen Packstücke packt als Lösungskomponente
    '''Ende der Funktionen für das Greedy Verfahren'''
    def print_packplan(self):
        for container in self.containers:
            container.save_packplan()
            container.save_container()
            
    # Visualisiere die Container in der Lösung
    def visualize(self):
        vis = o3d.visualization.Visualizer()
        vis.create_window()
        offset_x=0
        offset_x_delta=100
        for container in self.containers:
            for geo in container.get_geometry(offset_x=offset_x):
                vis.add_geometry(geo)
            
            offset_x+=offset_x_delta
            offset_x+=container.container.width

        o3d.visualization.ViewControl.set_zoom(vis.get_view_control(), 0.8)
        vis.run()
        vis.destroy_window()
        
    def __repr__(self):
        

        return "packing solution\n  " + "\n  ".join(str(x) for x in self.containers)

### Class: Population

In [111]:
class Population():

    def __init__(self, number_of_parents, population_size, parent_solutions, choosen_parent_pairing):
        self.pack_orders = [] # Liste in die alle Genome einer Population als Liste gespeichert werden
        self.mps_datasets = [] # Liste in der alle modifizierten Bestelldatensätze einer Population gespeichert werden
        self.parent_solutions = parent_solutions # Liste mit allen Lösungen der Populationseltern
        self.population_solutions = [] # Liste mit allen Lösungen der Population
        self.list_of_parent_pairs_for_crossover = [] # Liste mit Tupel für jedes Elternpaar, Tupel enthält nur Index des Elternteils
        self.number_of_parents = number_of_parents # Anzahl an Elternindividuen, die für den Crossover zu berücksichtigen sind
        self.population_size = population_size # Startpopulationsgröße
        self.choosen_parent_pairing = choosen_parent_pairing # Index für gewähltes Elternselektionsverfahren

    def add_pack_order(self, pack_order):
        self.pack_orders.append(pack_order)

    def add_mps_dataset(self, mps_dataset):
        self.mps_datasets.append(mps_dataset)

    def add_population_solution(self, solution):
        self.population_solutions.append(solution)

    def sort_population_solutions(self):
        self.population_solutions.sort(key = lambda solution: solution.usage_average, reverse = True) # Sortiere die Lösungen nach Volumenauslastung (größte Auslastung am Anfang der Liste)

    # Entferne alle Eltern, die nicht für die Rekombination berücksichtigt werden sollen
    def ceep_parents_for_pairing(self):
        self.parent_solutions = self.parent_solutions[:self.number_of_parents]


    '''Funktionen für die Berechnung der Elternpaarzuweisung'''
    # Bilde die Elternpaare für den Crossover anhand des ausgewählten Verfahrens
    def create_parent_pairs(self):
        if (self.choosen_parent_pairing == 0):
            self.get_parent_pairs_first_with_last()
        elif (self.choosen_parent_pairing == 1):
            self.get_parent_pairs_first_with_last_step_2()
        elif (self.choosen_parent_pairing == 2):
            self.get_parent_pairs_last_with_first()
        elif (self.choosen_parent_pairing == 3):
            self.get_parent_pairs_first_with_second()
        elif (self.choosen_parent_pairing == 4):
            self.get_parent_pairs_first_with_better_half()
        elif (self.choosen_parent_pairing == 5):
            self.get_parent_pairs_better_half_with_first()


    # Funktion zur Erzeugungvon Elternpaaren (nFWL) für den Crossover: Bester mit Schlechtestem, Zweitbester mit Zweitschlechtestem...
    def get_parent_pairs_first_with_last(self):

        # Initialisiere Zählervariable für die Auswahl des Elternteils A bzw. B
        parent_A = 0
        parent_B = 0
        # Initialisiere Zählervariable für die Auswahl des nächsten noch nicht berücksichtigten Elternteils
        parent_counter = 0

        while (parent_counter < self.number_of_parents/2): # Überprüfe, ob bereits alle Eltern berücksichtigt wurden
            parent_A = parent_counter # Weise Index für Elternteil A zu (Starte am Anfang der Liste mit allen Individuen)
            if ((self.number_of_parents%2 == 1) and ((self.number_of_parents-1)/2 == parent_counter)): # Überprüfe ob die Populationsgröße ungerade ist und alle Elternteile bis auf einen (den Mittleren) bereits berücksichtigt wurden
                parent_B = 0 # Weiße Index für Elternteil B zu: erstes Elternteil der Population
            else:
                parent_B = (self.number_of_parents - 1 - parent_counter) # Weise Index für Elternteil B zu (Starte am Ende der Liste mit allen Individuen)
            
            
            self.list_of_parent_pairs_for_crossover.append((parent_A, parent_B)) # Füger das erzeugte Elternpaar der Liste hinzu, welche alle Elternpaare enthält die miteinander rekombiniert werden sollen
            parent_counter += 1 # Erhöhe Zählervariable für die Auswahl des nächsten noch nicht berücksichtigten Elternteils


    # Funktion zur Erzeugungvon Elternpaaren (nFWLS2) für den Crossover: Bester mit Schlechtestem, Drittbester mit Drittschlechtestem...
    def get_parent_pairs_first_with_last_step_2(self):

        # Initialisiere Zählervariable für die Auswahl des Elternteils A bzw. B
        parent_A = 0
        parent_B = 0
        # Initialisiere Zählervariable für die Auswahl des nächsten noch nicht berücksichtigten Elternteils
        parent_counter = 0

        while (parent_counter < self.number_of_parents): # Überprüfe, ob bereits alle Eltern berücksichtigt wurden

            parent_A = parent_counter # Weise Index für Elternteil A zu (Starte am Anfang der Liste mit allen Individuen)

            if ((self.number_of_parents%2 == 1) and ((self.number_of_parents-1)/2 == parent_counter)): # Überprüfe ob die Populationsgröße ungerade ist und alle Elternteile bis auf einen (den Mittleren) bereits berücksichtigt wurden
                parent_B = 0 # Weiße Index für Elternteil B zu: erstes Elternteil der Population
            else:
                parent_B = (self.number_of_parents - 1 - parent_counter) # Weise Index für Elternteil B zu (Starte am Ende der Liste mit allen Individuen)
            
            
            self.list_of_parent_pairs_for_crossover.append((parent_A, parent_B)) # Füger das erzeugte Elternpaar der Liste hinzu, welche alle Elternpaare enthält die miteinander rekombiniert werden sollen
            parent_counter += 2 # Erhöhe Zählervariable für die Auswahl des nächsten noch nicht berücksichtigten Elternteils

    # Funktion zur Erzeugungvon Elternpaaren (LWF) für den Crossover: Schlechtester mit Bestem, Zweitschlechtester mit Zweitbestem...
    def get_parent_pairs_last_with_first(self):

        # Initialisiere Zählervariable für die Auswahl des Elternteils A bzw. B
        parent_A = 0
        parent_B = 0
        # Initialisiere Zählervariable für die Auswahl des nächsten noch nicht berücksichtigten Elternteils
        parent_counter = 0

        while (parent_counter < self.number_of_parents/2): # Überprüfe, ob bereits alle Eltern berücksichtigt wurden

            parent_A = parent_counter # Weise Index für Elternteil A zu (Starte am Anfang der Liste mit allen Individuen)

            if ((self.number_of_parents%2 == 1) and ((self.number_of_parents-1)/2 == parent_counter)): # Überprüfe ob die Populationsgröße ungerade ist und alle Elternteile bis auf einen (den Mittleren) bereits berücksichtigt wurden
                parent_B = 0 # Weiße Index für Elternteil B zu: erstes Elternteil der Population
            else:
                parent_B = (self.number_of_parents - 1 - parent_counter) # Weise Index für Elternteil B zu (Starte am Ende der Liste mit allen Individuen)
            
            
            self.list_of_parent_pairs_for_crossover.append((parent_B, parent_A)) # Füger das erzeugte Elternpaar der Liste hinzu, welche alle Elternpaare enthält die miteinander rekombiniert werden sollen
            parent_counter += 1 # Erhöhe Zählervariable für die Auswahl des nächsten noch nicht berücksichtigten Elternteils
        

    # Funktion zur Erzeugungvon Elternpaaren (TNB) für den Crossover: Bester mit Zweitbestem, Drittbester mit Viertbestem...
    def get_parent_pairs_first_with_second(self):

        # Initialisiere Zählervariable für die Auswahl des Elternteils A bzw. B
        parent_A = 0
        parent_B = 0
        # Initialisiere Zählervariable für die Auswahl des nächsten noch nicht berücksichtigten Elternteils
        parent_counter = 0

        while (parent_counter < self.number_of_parents): # Überprüfe, ob bereits alle Eltern berücksichtigt wurden

            parent_A = parent_counter # Weise Index für Elternteil A zu (Starte am Anfang der Liste mit allen Individuen)

            if ((self.number_of_parents%2 == 1) and (parent_counter == self.number_of_parents - 1)): # Überprüfe ob die Populationsgröße ungerade ist und alle Elternteile bis auf einen (den Letzten) bereits berücksichtigt wurden
                parent_B = 0 # Weiße Index für Elternteil B zu: erstes Elternteil der Population
            else:
                parent_B = (parent_counter + 1) # Weise Index für Elternteil B zu
            
            
            self.list_of_parent_pairs_for_crossover.append((parent_A, parent_B)) # Füger das erzeugte Elternpaar der Liste hinzu, welche alle Elternpaare enthält die miteinander rekombiniert werden sollen
            parent_counter += 2 # Erhöhe Zählervariable für die Auswahl des nächsten noch nicht berücksichtigten Elternteils


    # Funktion zur Erzeugungvon Elternpaaren (FWBH) für den Crossover: Bester mit besserer Häfte
    def get_parent_pairs_first_with_better_half(self):

        # Initialisiere Zählervariable für die Auswahl des Elternteils A bzw. B
        parent_A = 0
        parent_B = 0
        # Initialisiere Zählervariable für die Auswahl des nächsten noch nicht berücksichtigten Elternteils
        parent_counter = 0

        while (parent_counter < self.number_of_parents/2): # Überprüfe, ob bereits alle Eltern berücksichtigt wurden
            
            parent_A = 0 # Weise Index für Elternteil A zu (Betse Lösung)
            parent_B = parent_counter + 1     
            
            self.list_of_parent_pairs_for_crossover.append((parent_A, parent_B)) # Füger das erzeugte Elternpaar der Liste hinzu, welche alle Elternpaare enthält die miteinander rekombiniert werden sollen
            parent_counter += 1 # Erhöhe Zählervariable für die Auswahl des nächsten noch nicht berücksichtigten Elternteils


    # Funktion zur Erzeugungvon Elternpaaren (BHWF) für den Crossover: besere Hälfte mit Bestem
    def get_parent_pairs_better_half_with_first(self):

        # Initialisiere Zählervariable für die Auswahl des Elternteils A bzw. B
        parent_A = 0
        parent_B = 0
        # Initialisiere Zählervariable für die Auswahl des nächsten noch nicht berücksichtigten Elternteils
        parent_counter = 0

        while (parent_counter < self.number_of_parents/2): # Überprüfe, ob bereits alle Eltern berücksichtigt wurden
            
            parent_A = parent_counter + 1 # Weise Index für Elternteil A zu 
            parent_B = 0 # Weise Index für Elternteil B zu (Betse Lösung)
            
            
            self.list_of_parent_pairs_for_crossover.append((parent_A, parent_B)) # Füger das erzeugte Elternpaar der Liste hinzu, welche alle Elternpaare enthält die miteinander rekombiniert werden sollen
            parent_counter += 1 # Erhöhe Zählervariable für die Auswahl des nächsten noch nicht berücksichtigten Elternteils
    '''Ende der Funktionen für die Berechnung der Elternpaarzuweisung'''



    '''Funktionen für die Verschiedenen Crossovervarianten'''
    # Crossovervariante 0: Erzeuge 2 Nachkommen aus 2 Eltern
    # 1. Nachkommen: Erste gepackte Containerhälfte beider Eltern zuerst
    # 2. Nachkommen: Zweite gepackte Containerhälfte beider Eltern zuerst
    def create_two_offsprings_by_first_second_package_half_first(self, parent_A, parent_B, pack_order_offspring_A, pack_order_offspring_B):
        
        # Erzeuge Nachkommen A

        # wähle die als erstes gepackte Hälfte der Container in Elternteil A und dann B aus und Füge deren Packreihenfolge in die Packreihenfolge des Nachkommens ein 
        # (ist die gesamte Containerzahl ungerade wird aufgerundet z.B. Containeranzahl = 7, berücksichtige erste 4 Conatiner)
        pack_order_offspring_A = self.pick_first_half_of_parent_pack_order(self.parent_solutions[parent_A], pack_order_offspring_A)
        pack_order_offspring_A = self.pick_first_half_of_parent_pack_order(self.parent_solutions[parent_B], pack_order_offspring_A)

        # wähle die zweite Hälfte der gepackten Container in Elternteil A und dann B aus und Füge deren Packreihenfolge in die Packreihenfolge des Nachkommens ein 
        # (ist die gesamte Containerzahl ungerade wird aufgerundet z.B. Containeranzahl = 7, berücksichtige ab Conatiner 5)
        pack_order_offspring_A = self.pick_second_half_of_parent_pack_order(self.parent_solutions[parent_A], pack_order_offspring_A)
        pack_order_offspring_A = self.pick_second_half_of_parent_pack_order(self.parent_solutions[parent_B], pack_order_offspring_A)

        # Erzeuge Nachkommen B

        # wähle die zweite Hälfte der gepackten Container in Elternteil A und dann B aus und Füge deren Packreihenfolge in die Packreihenfolge des Nachkommens ein 
        pack_order_offspring_B = self.pick_second_half_of_parent_pack_order(self.parent_solutions[parent_A], pack_order_offspring_B)
        pack_order_offspring_B = self.pick_second_half_of_parent_pack_order(self.parent_solutions[parent_B], pack_order_offspring_B)

        # wähle die als erstes gepackte Hälfte der Container in Elternteil A und dann B aus und Füge deren Packreihenfolge in die Packreihenfolge des Nachkommens ein 
        pack_order_offspring_B = self.pick_first_half_of_parent_pack_order(self.parent_solutions[parent_A], pack_order_offspring_B)
        pack_order_offspring_B = self.pick_first_half_of_parent_pack_order(self.parent_solutions[parent_B], pack_order_offspring_B)

        return pack_order_offspring_A, pack_order_offspring_B

    # Crossovervariante 1: Erzeuge 2 Nachkommen aus 2 Eltern
    # Gepackte Container deren Auslastung > der durchschnittlichen Auslastung der Lösung sind zuerst
    def create_two_offsprings_by_package_half_with_highest_volume_usage_first(self, parent_A, parent_B, pack_order_offspring_A, pack_order_offspring_B):
        
        # Erzeuge Nachkommen A

        # wähle die die Container in Elternteil A bzw. B aus, deren Auslastung über der durchschnittlichen Auslastung des Elternteils liegt aus und Füge deren Packreihenfolge in die Packreihenfolge des Nachkommens ein
        pack_order_offspring_A = self.pick_containers_above_average_volume_usage_of_parent_pack_order(self.parent_solutions[parent_A], pack_order_offspring_A)
        pack_order_offspring_A = self.pick_containers_above_average_volume_usage_of_parent_pack_order(self.parent_solutions[parent_B], pack_order_offspring_A)

        # wähle die die Container in Elternteil A bzw. B aus, deren Auslastung unterhalb der durchschnittlichen Auslastung des Elternteils liegt aus und Füge deren Packreihenfolge in die Packreihenfolge des Nachkommens ein
        pack_order_offspring_A = self.pick_containers_below_average_volume_usage_of_parent_pack_order(self.parent_solutions[parent_A], pack_order_offspring_A)
        pack_order_offspring_A = self.pick_containers_below_average_volume_usage_of_parent_pack_order(self.parent_solutions[parent_B], pack_order_offspring_A)

        # Erzeuge Nachkommen B

        # wähle die die Container in Elternteil B bzw. A aus, deren Auslastung über der durchschnittlichen Auslastung des Elternteils liegt aus und Füge deren Packreihenfolge in die Packreihenfolge des Nachkommens ein
        pack_order_offspring_B = self.pick_containers_above_average_volume_usage_of_parent_pack_order(self.parent_solutions[parent_B], pack_order_offspring_B)
        pack_order_offspring_B = self.pick_containers_above_average_volume_usage_of_parent_pack_order(self.parent_solutions[parent_A], pack_order_offspring_B)

        # wähle die die Container in Elternteil B bzw. A aus, deren Auslastung unterhalb der durchschnittlichen Auslastung des Elternteils liegt aus und Füge deren Packreihenfolge in die Packreihenfolge des Nachkommens ein
        pack_order_offspring_B = self.pick_containers_below_average_volume_usage_of_parent_pack_order(self.parent_solutions[parent_B], pack_order_offspring_B)
        pack_order_offspring_B = self.pick_containers_below_average_volume_usage_of_parent_pack_order(self.parent_solutions[parent_A], pack_order_offspring_B)

        return pack_order_offspring_A, pack_order_offspring_B

    # Crossovervariante 2: Erzeuge 2 Nachkommen aus 2 Eltern
    # 1. Nachkommen: Erste gepackte Containerhälfte beider Elternteile zuerst
    def create_two_offsprings_by_first_package_half_first(self, parent_A, parent_B, pack_order_offspring_A, pack_order_offspring_B):
        
        # Erzeuge Nachkommen A

        # wähle die als erstes gepackte Hälfte der Container in Elternteil A und dann B aus und Füge deren Packreihenfolge in die Packreihenfolge des Nachkommens ein 
        # (ist die gesamte Containerzahl ungerade wird aufgerundet z.B. Containeranzahl = 7, berücksichtige erste 4 Conatiner)
        pack_order_offspring_A = self.pick_first_half_of_parent_pack_order(self.parent_solutions[parent_A], pack_order_offspring_A)
        pack_order_offspring_A = self.pick_first_half_of_parent_pack_order(self.parent_solutions[parent_B], pack_order_offspring_A)

        # wähle die zweite Hälfte der gepackten Container in Elternteil A und dann B aus und Füge deren Packreihenfolge in die Packreihenfolge des Nachkommens ein 
        # (ist die gesamte Containerzahl ungerade wird aufgerundet z.B. Containeranzahl = 7, berücksichtige ab Conatiner 5)
        pack_order_offspring_A = self.pick_second_half_of_parent_pack_order(self.parent_solutions[parent_A], pack_order_offspring_A)
        pack_order_offspring_A = self.pick_second_half_of_parent_pack_order(self.parent_solutions[parent_B], pack_order_offspring_A)

        # Erzeuge Nachkommen B

        # wähle die als erstes gepackte Hälfte der Container in Elternteil B und dann A aus und Füge deren Packreihenfolge in die Packreihenfolge des Nachkommens ein (ist die gesamte Containerzahl ungerade wird aufgerundet z.B. Containeranzahl = 7, berücksichtige erste 4 Conatiner)
        pack_order_offspring_B = self.pick_first_half_of_parent_pack_order(self.parent_solutions[parent_B], pack_order_offspring_A)
        pack_order_offspring_B = self.pick_first_half_of_parent_pack_order(self.parent_solutions[parent_A], pack_order_offspring_A)

        # wähle die zweite Hälfte der gepackten Container in Elternteil B und dann A aus und Füge deren Packreihenfolge in die Packreihenfolge des Nachkommens ein (ist die gesamte Containerzahl ungerade wird aufgerundet z.B. Containeranzahl = 7, berücksichtige ab Conatiner 5)
        pack_order_offspring_B = self.pick_second_half_of_parent_pack_order(self.parent_solutions[parent_B], pack_order_offspring_A)
        pack_order_offspring_B = self.pick_second_half_of_parent_pack_order(self.parent_solutions[parent_A], pack_order_offspring_A)

        return pack_order_offspring_A, pack_order_offspring_B
    '''Ende Funktionen für die Verschiedenen Crossovervarianten'''


    '''Sub-Funktionen für die Verschiedenen Crossovervarianten'''
    '''Subfunktionen der Crossovervariante 0 & 2'''
    # Crossovervariante 0 & 2: Zieht die Packreihenfolge von der als erstes gepackten Conatinerhälfte heran und fügt sie der Packreihenfolge des Nachkommen hinzu
    def pick_first_half_of_parent_pack_order(self, parent, pack_order_offspring):
        
        # Initialisiere Anzahl Container des Elternteils
        conatiners_in_parent = parent.container_number
        # Initialisiere Zählervariable für die Auswahl eines spezifischen Containers des Elternteils (Starte mit erstem Container der ersten Hälfte)
        picked_container_in_parent = 0
        
        # wähle die als erstes gepackte Hälfte der Container im Elternteil aus und Füge deren Packreihenfolge in die neu zu erzeugende Packreihenfolge ein (ist die gesamte Containerzahl ungerade wird aufgerundet z.B. Containeranzahl = 7, berücksichtige erste 4 Conatiner)
        while (picked_container_in_parent < (conatiners_in_parent/2)): 
            # Durchlaufe alle Packstücke des gewählten Containers aus dem Elternteil
            for article in parent.containers[picked_container_in_parent].article_locations:
                if (article.package_ID not in pack_order_offspring): # Überprüfe ob der Artikel noch nicht in der Packreihenfolge dieses Nachkommen vorkommt
                    pack_order_offspring.append(article.package_ID) # und füge die Packstück ID aller Packstücke dieses Containers der Reihe nach der Packstückreihenfolge des Nachkommen hinzu

            picked_container_in_parent += 1 # Erhöhe Zählervariable für die Containerauswahl des Elternteils
        
        # Gebe die erzeugte (Teil-)Packreihenfolge des Nachkommens zurück
        return pack_order_offspring

    # Crossovervariante 0 & 2: Zieht die Packreihenfolge von der als letztes gepackten Conatinerhälfte heran und fügt sie der Packreihenfolge des Nachkommen hinzu
    def pick_second_half_of_parent_pack_order(self, parent, pack_order_offspring):
        
        # Initialisiere Anzahl Container des Elternteils
        conatiners_in_parent = parent.container_number
        # Initialisiere Zählervariable für die Auswahl eines spezifischen Containers des Elternteils (Starte mit erstem Container der zweiten Hälfte)
        picked_container_in_parent = round(conatiners_in_parent/2) # aufrunden für ungerade Containeranzahl in Elternteil (ist die gesamte Containerzahl ungerade wird aufgerundet z.B. Containeranzahl = 7, berücksichtige ab Conatiner 5)
        
        # wähle die zweite Hälfte der gepackten Container im Elternteil aus und Füge deren Packreihenfolge in die neu zu erzeugende Packreihenfolge ein 
        while (picked_container_in_parent < conatiners_in_parent): 
            # Durchlaufe alle Packstücke des gewählten Containers aus dem Elternteil
            for article in parent.containers[picked_container_in_parent].article_locations:
                if (article.package_ID not in pack_order_offspring): # Überprüfe ob der Artikel noch nicht in der Packreihenfolge dieses Nachkommen vorkommt
                    pack_order_offspring.append(article.package_ID) # und füge die Packstück ID aller Packstücke dieses Containers der Reihe nach der Packstückreihenfolge des Nachkommen hinzu

            picked_container_in_parent += 1 # Erhöhe Zählervariable für die Containerauswahl des Elternteils
        
        # Gebe die erzeugte (Teil-)Packreihenfolge des Nachkommens zurück
        return pack_order_offspring
    '''Ende der Subfunktionen der Crossovervariante 0 & 2'''

    '''Subfunktionen der Crossovervariante 1'''
    # Crossovariante 1: Zieht die Packreihenfolge der Conatiner heran, deren Auslastung höher als die durchschnittliche Auslastung des übergebenen Elterteils ist und fügt sie der Packreihenfolge des Nachkommen hinzu
    def pick_containers_above_average_volume_usage_of_parent_pack_order(self, parent, pack_order_offspring):
        
        # Initialisiere Anzahl Container des Elternteils
        conatiners_in_parent = parent.container_number
        # Initialisiere die durchschnittliche Auslastung des Elternteils
        average_usage_of_parent = parent.usage_average
        # Initialisiere Liste die die (restlichen) Container des übergebenen Elternteils enthält
        parent_containers = copy.deepcopy(parent)
        
        # Durchlaufe dafür alle Container so oft, wie es noch einen Container gibt, der noch nicht der Packreihenfolge des Nachkommens hinzugefügt wurde & dessen Auslastung höher ist als die durchschnittliche Auslastung des Elternteils
        for nr_of_container_iterations in range(conatiners_in_parent):

            # Initialisiere Variable zum Speichern der Auslastung des Containers mit der höchsten Auslastung, der noch nicht der Packreihenfolge des Nachkommens hinzugefügt wurde
            volume_usage_of_best_container = 0
            # Initialisiere Variable zum Speichern des Index des Containers mit der höchsten Auslastung, der noch nicht der Packreihenfolge des Nachkommens hinzugefügt wurde
            index_of_best_container = -1
            # Initialisiere Variable zum Speichern der Auslastung eines ausgewählten Containers
            volume_usage_of_selected_container = 0
            # Initialisiere die Anzhal an Conatainern des Elternteils, die noch nicht der Packreihenfolge des Nachkommens hinzugefügt wurden
            not_packed_conatiners_in_parent = len(parent_containers.containers)

            # Durchlaufe alle Container, die noch nicht der Packreihenfolge des Nachkommens hinzugefügt wurden und finde davon den Container mit der höchsten Auslastung
            for container_index in range(not_packed_conatiners_in_parent):
                volume_usage_of_selected_container = parent_containers.containers[container_index].volume_usage_rate # Speichere die Volumenauslastung des aktuell betrachteten Containers
                if (volume_usage_of_selected_container > volume_usage_of_best_container): # Überprüfe, ob die Volumenasulastung des aktuell betrachteten Containers der bisher größten erfassten Auslastung entspricht
                    volume_usage_of_best_container = volume_usage_of_selected_container # Speicher die Auslastung dieses Containers als die bisher größte erfasste Auslastung
                    index_of_best_container = container_index # Speicher den Index dieses Containers
                
                # Erhöhe den Container Index um eins, um die Schleife für den Nächsten Container erneut zu starten
                container_index += 1
            
            # Nachdem der Container mit der größten Auslastung identifirt ist, der noch nicht der Packreihenfolge des Nachkommens hinzugefügt wurde:
            if (volume_usage_of_best_container >= average_usage_of_parent): # Überprüfe, ob dessen Auslastung höher ist als die durchschnittliche Auslastung des Elternteils
                # Durchlaufe alle Packstücke des gewählten Containers aus dem Elternteil
                for article in parent_containers.containers[index_of_best_container].article_locations:
                    if (article.package_ID not in pack_order_offspring): # Überprüfe ob der Artikel noch nicht in der Packreihenfolge dieses Nachkommen vorkommt
                        pack_order_offspring.append(article.package_ID) # und füge die Packstück ID aller Packstücke dieses Containers der Reihe nach der Packstückreihenfolge des Nachkommen hinzu

                # Entferne den gewählten Container aus der Containermenge des Elternteils
                del parent_containers.containers[index_of_best_container]
            
            else: # Bereits alle Container, deren Auslastung größer als die durchschnittlichen Volumenauslatung des Elternteils ist wurden der Packreihenfolge des Nachkommens hinzugefügt
                break # Unterbreche die Suche nach weiteren Containern
            
            # Erhöhe die Anzahl der durchgeführten Durchsuchungen aller Conatiner nach einem Container, der noch nicht der Packreihenfolge des Nachkommens hinzugefügt wurde & dessen Auslastung höher ist als die durchschnittliche Auslastung des Elternteils
            nr_of_container_iterations += 1
        
        # Gebe die erzeugte (Teil-)Packreihenfolge des Nachkommens zurück
        return pack_order_offspring
        
    # Crossovervariante 1: Zieht die Packreihenfolge der Conatiner heran, deren Auslastung geringer als die durchschnittliche Auslastung des übergebenen Elterteils ist und fügt sie der Packreihenfolge des Nachkommen hinzu
    def pick_containers_below_average_volume_usage_of_parent_pack_order(self, parent, pack_order_offspring):
        
        # Initialisiere Anzahl Container des Elternteils
        conatiners_in_parent = parent.container_number
        # Initialisiere die durchschnittliche Auslastung des Elternteils
        average_usage_of_parent = parent.usage_average
        # Initialisiere Liste die die (restlichen) Container des übergebenen Elternteils enthält
        parent_containers = copy.deepcopy(parent)
        
        # Durchlaufe dafür alle Container so oft, wie es noch einen Container gibt, der noch nicht der Packreihenfolge des Nachkommens hinzugefügt wurde & dessen Auslastung höher ist als die durchschnittliche Auslastung des Elternteils
        for nr_of_container_iterations in range(conatiners_in_parent):

            # Initialisiere Variable zum Speichern der Auslastung des Containers mit der höchsten Auslastung unterhalb des Auslastungsdurchschnitts, der noch nicht der Packreihenfolge des Nachkommens hinzugefügt wurde
            volume_usage_of_best_container_below_average = 0
            # Initialisiere Variable zum Speichern des Index des Containers mit der höchsten Auslastung unterhalb des Auslastungsdurchschnitts, der noch nicht der Packreihenfolge des Nachkommens hinzugefügt wurde
            index_of_best_container_below_average = -1
            # Initialisiere Variable zum Speichern der Auslastung eines ausgewählten Containers
            volume_usage_of_selected_container = 0
            # Initialisiere die Anzhal an Conatainern des Elternteils, die noch nicht der Packreihenfolge des Nachkommens hinzugefügt wurden
            not_packed_conatiners_in_parent = len(parent_containers.containers)

            # Durchlaufe alle Container, die noch nicht der Packreihenfolge des Nachkommens hinzugefügt wurden und finde davon den Container mit der höchsten Auslastung unterhalb des Auslastungsdurchschnitts
            for container_index in range(not_packed_conatiners_in_parent):
                volume_usage_of_selected_container = parent_containers.containers[container_index].volume_usage_rate # Speichere die Volumenauslastung des aktuell betrachteten Containers
                if (volume_usage_of_selected_container > volume_usage_of_best_container_below_average and volume_usage_of_selected_container < average_usage_of_parent): # Überprüfe, ob die Volumenasulastung des aktuell betrachteten Containers der bisher größten erfassten Auslastung unterhalb des Auslastungsdurchschnitts entspricht
                    volume_usage_of_best_container_below_average = volume_usage_of_selected_container # Speicher die Auslastung dieses Containers als die bisher größte erfasste Auslastung unterhalb des Auslastungsdurchschnitts
                    index_of_best_container_below_average = container_index # Speicher den Index dieses Containers
                
                # Erhöhe den Container Index um eins, um die Schleife für den Nächsten Container erneut zu starten
                container_index += 1
            
            # Nachdem der Container mit der größten Auslastung unterhalb des Auslastungsdurchschnitts identifirt ist, der noch nicht der Packreihenfolge des Nachkommens hinzugefügt wurde:
            if (volume_usage_of_best_container_below_average > 0): # Überprüfe, ob es überhaupt noch solch einen Conatiner gab
                # Durchlaufe alle Packstücke des gewählten Containers aus dem Elternteil
                for article in parent_containers.containers[index_of_best_container_below_average].article_locations:
                    if (article.package_ID not in pack_order_offspring): # Überprüfe ob der Artikel noch nicht in der Packreihenfolge dieses Nachkommen vorkommt
                        pack_order_offspring.append(article.package_ID) # und füge die Packstück ID aller Packstücke dieses Containers der Reihe nach der Packstückreihenfolge des Nachkommen hinzu

                # Entferne den gewählten Container aus der Containermenge des Elternteils
                del parent_containers.containers[index_of_best_container_below_average]
            
            else: # Bereits alle Container, deren Auslastung kleiner als die durchschnittlichen Volumenauslatung des Elternteils ist wurden der Packreihenfolge des Nachkommens hinzugefügt
                break # Unterbreche die Suche nach weiteren Containern
            
            # Erhöhe die Anzahl der durchgeführten Durchsuchungen aller Conatiner nach einem Container, der noch nicht der Packreihenfolge des Nachkommens hinzugefügt wurde & dessen Auslastung geringer ist als die durchschnittliche Auslastung des Elternteils
            nr_of_container_iterations += 1
        
        # Gebe die erzeugte (Teil-)Packreihenfolge des Nachkommens zurück
        return pack_order_offspring
    '''Ende der Subfunktionen der Crossovervariante 1'''
    '''Ende der Sub-Funktionen für die Verschiedenen Crossovervarianten'''



    '''Funktionen für die Mutation'''
    # Bekommt die Packreihenfolge aller Nachkommen übergeben und mutierte diese solange bis eine noch nicht vorgekommene Packreihenfolge entsteht
    def mutation(self, tabu_list):
        for offspring_index in range(len(self.pack_orders)): # Durchlaufe die Packreihenfolge aller Nachkommen

            self.pack_orders[offspring_index] = self.change_2_random_articles(self.pack_orders[offspring_index]) # Führe Funktion für Mutation auf: tauschen 2er zufällig gewählter Artikel aus
            
            if (self.check_if_pack_order_already_used(self.pack_orders[offspring_index], tabu_list) == True): # Mutiere so lange weiter, bis ein noch nicht verwendete Packreihenfolge entsteht 
                
                self.pack_orders[offspring_index] = self.change_2_random_articles(self.pack_orders[offspring_index])
    # Tausche zwei zufällig gewählte Artikel in der übergebenen Packreihenfolge
    def change_2_random_articles(self, pack_order):
        # Initialisiere Variable zum Zwischenspeichern der Artikel ID, des zu tauschenden Artikels
        article_ID_of_article_1 = 0

        # Initialisiere Index Variable für die beiden zu tauschenden Artikel. Wähle dabei den Index zwischen 0 und der Anzahl an Artikeln in dieser Packreihenfolge
        index_articel_1 = random.randrange(0,len(pack_order))
        index_articel_2 = random.randrange(0,len(pack_order))
        article_ID_of_article_1 = pack_order[index_articel_1] # Speichere die Artikel ID des ersten Artikels zwischen
        pack_order[index_articel_1] = pack_order[index_articel_2] # Überschreibe die Position des ersten Artikels mit der Artikel ID des zweiten Artikels
        pack_order[index_articel_2] = article_ID_of_article_1 # Überschreibe die Position des zweiten Artikels mit der Artikel ID des ersten Artikels
    
        return pack_order

    # Funktion die überprüft, ob die übergebene PAckreihenfolge bereits verwendet wurden. Durch Abgleich mit der Tabu-Liste
    def check_if_pack_order_already_used(self, pack_order, tabu_list):
        pack_order_already_used = pack_order in tabu_list # Überprüfe, ob die übergeben Packreihenfolge bereits berechnet wurde, falls ja: True
        return pack_order_already_used
    '''Ende der Funktionen für die Mutation'''



    '''Funktionen für die Erzeugung von Packreihenfolgen'''
    # Erzeuge die Packreihenfolgen für die Startpopulation
    def create_start_population_pack_orders(self, article_count):   
        # Erzeuge eine Liste, welche die Artikel ID jedes Packstücks enthält, wie sie in den MPS enthalten ist. Also beginnend mit 1
        all_article_IDs = [article_ID for article_ID in range(1, (article_count + 1))]

        self.add_pack_order(all_article_IDs) # Füge die Packreihenfolge des initialen MPS Datensatzes hinzu
        
        for i in range(self.population_size):
            random.seed(article_count+i) # Setze einen Seed für die Reproduzierbarkeit der erzeugten Reihenfolge, Seed muss sich pro Durchgang ädnern, damit unterschiedliche Reihenfolgen erzeugt werden
            self.add_pack_order(random.sample(all_article_IDs, len(all_article_IDs))) # Füge erzeugte gemischte Packreihenfolge der Liste für alle Reihenfolgen hinzu
    
    # Funktion zum Erzeugen der Packreihenfolge einer Nachkommen-Population
    def create_packing_orders_for_current_offspring_generation(self, tabu_list, list_of_choosen_crossovers):
        '''Starte Selektion'''

        # Selektiere Eltern die für die Rekombination berücksichtigt werden sollen
        self.ceep_parents_for_pairing()

        '''Ende Selektion'''


        '''Starte Crossover'''

        # Bilde die Elternpaare für den Crossover anhand des ausgewählten Verfahrens
        self.create_parent_pairs()

        # Initialisiere Index zum iterieren der Liste für die durchzuführenden Crossovervarianten
        list_index = 0

        # Führe einen Crossover für jedes Elternpaar für jede festgelegte Crossovervariante durch:
        while(list_index < len(list_of_choosen_crossovers)): # Jedes Elternpaar wird mit in der Liste definierten Crossover Varianten rekombiniert
            
            nr_of_crossover_variant = list_of_choosen_crossovers[list_index] # Weise Crossovervariante zu, welche diese Schleifeniteration durchgeführt werden soll

            # Führe für jedes Elternpaar die definierte Corssovervariante durch
            for parent_pair in self.list_of_parent_pairs_for_crossover:
                parent_A = parent_pair[0] # Weise den Index für Elternteil A zu
                parent_B = parent_pair[1] # Weise den Index für Elternteil B zu

                # Initialisiere Listen, in welche die Packreihenfolgen der beiden Nachkommen gespeichert werden
                pack_order_offspring_A = []
                pack_order_offspring_B = []

                if (nr_of_crossover_variant == 0): # 0. Crossovervariante

                    pack_order_offspring_A, pack_order_offspring_B = self.create_two_offsprings_by_first_second_package_half_first(parent_A, parent_B, pack_order_offspring_A, pack_order_offspring_B)

                elif (nr_of_crossover_variant == 1): # 1. Crossovervariante
                    
                    pack_order_offspring_A, pack_order_offspring_B = self.create_two_offsprings_by_package_half_with_highest_volume_usage_first(parent_A, parent_B, pack_order_offspring_A, pack_order_offspring_B)

                elif (nr_of_crossover_variant == 2): # 2. Crossovervariante

                    pack_order_offspring_A, pack_order_offspring_B = self.create_two_offsprings_by_first_package_half_first(parent_A, parent_B, pack_order_offspring_A, pack_order_offspring_B)

                else: # In der Liste für die durchzuführenden Crossovervarianten wurde eine Zahl aufgenommen, für die keine Crossovervariante definiert ist
                    print("!!!!!!! Die Liste für Crossovervarianten enthält einen nicht definierten Wert. !!!!!!!")
                    break
                self.add_pack_order(pack_order_offspring_A) # Füge die neue erzeugte Packreihenfolge des Nachkommens A der Liste aller Nachkommen hinzu
                self.add_pack_order(pack_order_offspring_B) # Füge die neue erzeugte Packreihenfolge des Nachkommens B der Liste aller Nachkommen hinzu

            list_index += 1 # Erhöhe Index zum iterieren der Liste für die durchzuführenden Crossovervarianten

        '''Ende Crossover'''


        '''Start Mutation'''

        self.mutation(tabu_list) # Mutiere alle Packreihenfolgen und weiße sie der Populationsinstanz zu
           
        '''Ende Mutation'''
    '''Ende Funktionen für die Erzeugung von Packreihenfolgen'''



    '''Funktionen für die Erzeugung von MPS Datensätzen'''
    # Erzeuge die MPS Datensätze für alle Packreihenfolgen der Startpopulation
    def create_start_population_mps(self, initial_modified_order_data):
        for pack_order in self.pack_orders: # Durchlaufe alle erzeugten Packreihenfolgen der ersten Population
            # Rufe Funktion auf, um MPS für die entsprechende Packreihenfolge zu erzeugen, übergebe initialen MPS Datensatz, Packreihenfolge des Individuums
            self.create_MPS_for_individual(initial_modified_order_data, pack_order)
       
    # Erzeuge die MPS Datensätze für alle Packreihenfolgen einer Nachkommen-Population
    def create_offspring_population_mps(self, initial_modified_order_data):
        for pack_order in self.pack_orders: # Durchlaufe alle Packreihenfolgen der Population
            # Rufe Funktion auf, um MPS für die entsprechende Packreihenfolge zu erzeugen, übergebe initialen MPS Datensatz, Packreihenfolge des Individuums
            self.create_MPS_for_individual(initial_modified_order_data, pack_order)

    # Erzeuge den MPS Datensatz für eine übergebene Packreihenfolge (Genotyp)
    def create_MPS_for_individual(self, initial_modified_order_data, pack_order_offspring):
        # Initialisiere eine leere Liste um den MPS Datensatz eines Nachkommens darin zu speichern
        mps_for_offspring = []
        for package_index in range(len(pack_order_offspring)): # Durchlaufe die Packstückreihenfolge des Nachkommen
            package_ID = pack_order_offspring[package_index] # Speichere die Packstück ID für das aktuell betrachtete Packstück in der Reihenfolge
            mapping_package_ID_to_list_index = package_ID -1 # Ziehe von der Packstück ID eins um den korrekten Listenindex zu haben
            mps_for_offspring.append(initial_modified_order_data[mapping_package_ID_to_list_index][:]) # Wähle durch den korrekten Listenindex die MPS Daten für das betrachtete Packstück aus dem initialen MPS Datensatz aus und Füge es der Liste für den MPS Datensatz des Nachkommens hinzu
        
        self.add_mps_dataset(mps_for_offspring)
        # return mps_for_offspring # Gebe den erzeugten MPS Datensatz für den Nachkommen zurück
    '''Ende Funktionen für die Erzeugung von MPS Datensätzen'''
    


    '''Funktionen für die Berechnung der Fitness'''
    # Liefere die Lösung für jede Populationsinstanz sortiert nach durchschnittlicher Containerauslastung (größte Auslatsung zuerst)
    def get_population_solutions(self, container_data, index_best_solution):
        # Initialisiere Liste, um die Volumenauslastung jeder Lösung zu speichern
        volume_usage_each_solution = []

        for modified_order_data in self.mps_datasets:
            solution = Solution(modified_order_data, container_data) # erzeugte einer leere Lösung und Initailisiere sie mit der MPS Packreihenfolge & den Containerdaten
            solution.calculate_solution() # Berechne die Lösung
            volume_usage_each_solution.append(solution.usage_average) # Speichere die Volumenauslastung in der Liste mit den Ausalstungen aller Lösungen
            
            self.add_population_solution(solution) # Speichere die Lösung mit ihrer Volumenauslastung in einer Liste

        # Berechen den Index der Lösung mit der höchsten Auslastung
        max_value = max(volume_usage_each_solution)
        max_index = volume_usage_each_solution.index(max_value)

        index_best_solution.append(max_index) # Füge den Index der Liste mit den Indizes der besten Lösung jeder Population hinzu  
        return index_best_solution
    '''Ende Funktionen für die Berechnung der Fitness'''


### Class: Genetic Algorithm

In [112]:
import itertools

class Genetic_Algorithm:
    def __init__(self, initial_modified_order_data, container_data, population_size, nr_of_iterations, choosen_parent_pairing, list_of_choosen_crossovers, nr_of_parents_to_keep):
        self.initial_modified_order_data = initial_modified_order_data
        self.container_data = container_data

        self.population_size = population_size # Initialisiere die Populationsgröße
        self.number_of_parents = population_size # Initialisiere die Anzahl an berücksichtigten Eltern
        self.nr_of_iterations = nr_of_iterations # Initialisiere die Anzahl der Iterationen
        self.choosen_parent_pairing = choosen_parent_pairing # Initialisiere Variable zur Festlegung der Paarbildungsvariante
        self.list_of_choosen_crossovers = list_of_choosen_crossovers # Initialisiere Liste zur Festlegung der Crossovervariante
        self.nr_of_parents_to_keep = nr_of_parents_to_keep # Initialisiere Variable zur Festlegung der Anzahl an Eltern, die bei der Populationsbildung aus der Vorgängerpopulation mitberücksichtigt werden sollen

        self.iteration_numbers = [] # Initialisiere eine Liste, welche die Iterationsnummer nacheinander speichert (dient ausschließlich der Visualisierung)
        self.best_volume_usage_values = [] # Initialisiere eine Liste, welche die Volumenauslastung der besten Lösung jeder Generation speichert (dient ausschließlich der Visualisierung)
        self.index_best_solution = list() # Initialisiere eine Liste, welche den Index des besten Individuums einer Population trägt (dient ausschließlich der Visualisierung)

        self.tabu_list = [] # Initialisiere eine Liste in der alle bereits verwendeten Packreihenfolgen gespeichert werden
        
    def add_iteration_number(self, iteration_number):
        self.iteration_numbers.append(iteration_number)

    def add_best_volume_usage(self, best_volume_usage):
        self.best_volume_usage_values.append(best_volume_usage)

    # Fügt eine übergebene Mange an Packreihenfolgen der Tabu-Liste hinzu fügt
    def add_pack_orders_to_tabu_list(self, pack_orders_current_population):
        for pack_order in pack_orders_current_population: # Durchlaufe die übergebenen Packreihenfolge
            self.tabu_list.append(copy.deepcopy(pack_order)) # Füge jede Packreihenfolge der Tabu Liste hinzu


    # Erzeugen & berechnen der ersten Population
    def run_initial_population(self):
            # Erzeuge eine leere Population
            initial_population = Population(self.number_of_parents, self.population_size-1, [], self.choosen_parent_pairing) # -1 bei Populationsgröße weil das durch die Basisheuristik erzeugte Individuum in die Startpopulation ebenfalls miteinfließt (wird später hinzugefügt)
            # Initialisiere die Anzahl der Packstücke
            article_count = len(self.initial_modified_order_data)

            # Erzuege die Packreihenfolgen für die erste Population
            initial_population.create_start_population_pack_orders(article_count) 
            
            # Füge die Packreihenfolgen der Liste mit allen bereits verwendeten Packreihenfolgen hinzu
            self.add_pack_orders_to_tabu_list(initial_population.pack_orders)

            # Erzeuge die MPS Datensätze für die erste Population
            initial_population.create_start_population_mps(self.initial_modified_order_data)


            # Berechne die Lösungen für jede MPS der ersten Population und sortiere sie nach Volumenauslastung (größte Auslastung am Anfang der Liste)
            # Füge den IIndex der besten erzeugten Lösung in die entsprechende Visualisierungsliste ein
            self.index_best_solution = initial_population.get_population_solutions(self.container_data, self.index_best_solution) 
     
            # Sortiere die Lösungen nach Volumenauslastung (größte Auslastung am Anfang der Liste)
            initial_population.sort_population_solutions()

            # Füge die Iteratiosnnummer und die beste Auslastung der ersten Generation in die entsprechenden Visualisierungslisten ein
            self.add_iteration_number(-self.nr_of_iterations)
            self.add_best_volume_usage(initial_population.population_solutions[0].usage_average)
            return initial_population


    # Erzeugen & berechnen aller Folgegenerationen
    def run_all_offspring_generations(self, initial_population):
        # Initialisiere die Vorgängerpopulation mit der initialen Population
        previous_population = initial_population

        '''Durchlaufe die definierte Anzahl an Generationen'''

        while (self.nr_of_iterations > 0):
            # Erzeuge eine leere Population
            current_population = Population(self.number_of_parents, self.population_size, previous_population.population_solutions, self.choosen_parent_pairing)
            '''Code für die Durchführung des GAs nur mit Mutation'''
            # # Weiße Packreihenfolge der vorherigen Generation der Packreihenfolge der aktuellen Generation zu
            # current_population.pack_orders = previous_population.pack_orders
            # # Führe nur Mutation durch
            # current_population.mutation(self.tabu_list) # Mutiere alle Packreihenfolgen und weiße sie der Populationsinstanz zu
            '''Ende des Codes für die Durchführung des GAs nur mit Mutation'''
            
            '''Abschnitt auskommentieren für nur Mutation'''
            # Zusätzlich muss der Funktionsaufruf für die Mutation innerhalb der Funktion create_packing_orders_for_current_offspring_generation in der Klasse Population auskommentiert werden
            # Erzeuge Packreihenfolgen für die neue Population
            current_population.create_packing_orders_for_current_offspring_generation(self.tabu_list, self.list_of_choosen_crossovers) # Erzuege die Packreihenfolgen für die neue Generation
            '''Ende des auskommentieren Abschnitts für nur Mutation'''

            '''Lokale Suche für die beste Lösung (Abschnitt auskommentieren für Berechnungen ohne lokale Suche)'''
            # Erzeuge Population nur mit bester Lösung der Vorgängerpopulation und mutiere sie für eine lokale Suche
            dummy_population = copy.deepcopy(previous_population) # Erzeuge Kopie der Vorgängerpopulation
            dummy_population.pack_orders = [dummy_population.pack_orders[0]] # Entferne alle Packreihenfolgen der Vorgängergeneration bis auf die beste
            dummy_population.mutation(self.tabu_list) # Mutiere die beste Packreohenfolge der Vorgängergeneration
            current_population.add_pack_order(dummy_population.pack_orders[0]) #Füge die mutierte Packreohenfolge den Packreihenfolgen der aktuellen Generation hinzu
            '''Ende der lokalen Suche für die beste Lösung'''

            # Füge die Packreihenfolgen der Liste mit allen bereits verwendeten Packreihenfolgen hinzu
            self.add_pack_orders_to_tabu_list(current_population.pack_orders)
            
            # Erzeuge die MPS Datensätze für die aktuelle Population
            current_population.create_offspring_population_mps(self.initial_modified_order_data)
            
            # Berechne die Lösung für jeden Nachkommen der aktuellen Population und sortiere die Lösungen nach der durchschnittlichen Volumenauslastung (größte Auslastung am Anfang der Liste)
            # Füge den Index der besten erzeugten Lösung in die entsprechende Visualisierungsliste ein
            self.index_best_solution = current_population.get_population_solutions(self.container_data, self.index_best_solution)
            
            '''Fortsetzung Lokale Suche für die beste Lösung (Abschnitt auskommentieren für Berechnungen ohne lokale Suche)'''
            # Prüfe ob der durch die lokale Suche erzeugte Mutant schlechter ist als die nicht mutierte beste Lösung der Vorgängergeneration.
            if current_population.population_solutions[-1].usage_average <= previous_population.population_solutions[0].usage_average: 
                current_population.population_solutions[-1] = previous_population.population_solutions[0] # Ersetze den Mutanten durch die beste Lösung der Vorgängergeneration
                current_population.pack_orders[-1] = previous_population.pack_orders[0] # Ersetze die Packreihenfolge des Mutanten durch die der besten Lösung der Vorgängergeneration
            '''Ende Fortsetzung der lokalen Suche für die beste Lösung'''
            
            '''Ohne lokale Suche (Abschnitt muss ausgeführt werden)'''
            # # Ergänze beste x Lösung der Eltern in der Liste
            # for parent_solution in itertools.islice(previous_population.population_solutions, 0, self.nr_of_parents_to_keep):
            #     current_population.add_population_solution(parent_solution)
            '''Ende ohne lokale Suche'''

            # Sortiere die Lösungen nach Volumenauslastung (größte Auslastung am Anfang der Liste)
            current_population.sort_population_solutions()
            self.nr_of_iterations -= 1 # Verringere die Zählervariable für die Anzahl an Generationen/ Iterationen, die der genetischen Algorithmus noch durchlaufen soll
            
            # Speichere die erzeugte Population als Vorgängerpopulation für die nächste zu erzeugende Population
            previous_population = current_population

            # Füge die Iteratiosnnummer und die beste Auslastung der ersten Generation in die entsprechenden Visualisierungslisten ein
            self.add_iteration_number(-self.nr_of_iterations)
            self.add_best_volume_usage(current_population.population_solutions[0].usage_average)

        '''Die definierte Anzahl an Generationen wurde durchlaufen'''
        return current_population.population_solutions[0]


    # Funktion die die Volumenauslastung über die Generationen & die beste erzeugte Lösung pro Population visualisiert
    def show_line_chart_with_best_solutions_of_each_generation(self):
        # Initialisiere die subplot Funktion
        figure, axis = plt.subplots(2, 1, figsize=(25,10))

        axis[0].plot(self.iteration_numbers, self.best_volume_usage_values)
        axis[0].set_title('Population size: ' + str(self.population_size) + ', Iterations: ' + str(self.nr_of_iterations) + ', Crossover: ' + str(self.list_of_choosen_crossovers).strip('[]') + ', Pairing Variant: ' + str(self.choosen_parent_pairing))
        axis[0].set_ylabel('Auslastung')

        axis[1].plot(self.iteration_numbers, self.index_best_solution)
        axis[1].set_xlabel('Populationsnummer')
        axis[1].set_ylabel('Index beste neue Lösung')

        plt.show()
    


    # Führe alle Iterationen des evolutionären Algorithmus durch
    def run_genetic_algorithm(self):     
        # Erzeuge und berechne die erste Population
        initial_population = self.run_initial_population()

        # Erzeuge und berechne die Populationen aller Folgegenerationen
        best_solution = self.run_all_offspring_generations(initial_population)


        # Erezuge einen line plot, der die Auslastung pro Generation anzeigt
        # self.show_line_chart_with_best_solutions_of_each_generation()

        # Gebe die finale Lösung zurück
        return best_solution, self.best_volume_usage_values, self.index_best_solution
        

### Class: Packing Algorithm

In [113]:
class Packing_Algorithm:

    def __init__(self, order_instance, population_size, nr_of_iterations, choosen_parent_pairing, list_of_choosen_crossovers, nr_of_parents_to_keep):
        self.start_time = time.time() # Anfangszeit
        self.end_time = 0 # Schlusszeit
        self.time_needed = 0 # benötigte Zeit

        # Dateinamen der Bestellung und der Containerliste festlegen
        self.order_file_name = order_instance + ".xlsx"
        self.container_file_name = "Containerliste" + ".xlsx"    

        # Initialisiere Parameter für den Evolutionären Algorithmus
        self.population_size = population_size # Startpopulationsgröße
        self.nr_of_iterations = nr_of_iterations # Anzahl an Folgegenerationen die der evolutionöre Algorithmus erzeugen soll
        self.choosen_parent_pairing = choosen_parent_pairing # Gewählte Elternselektion
        self.list_of_choosen_crossovers = list_of_choosen_crossovers # Gewähltes Rekombinationsverfahren
        self.nr_of_parents_to_keep = nr_of_parents_to_keep # Anzahl der besten Individuen, die in die Folgegeneration übernommen werden sollen. Hat nur Auswirkungen, wenn die lokale Suche in der Funktion run_all_offspring_generations der Klasse Populations nicht durchgeführt wird.
        
        # Rufe Datenvorverarbeitung auf und erhalte die initialen modifizierten Bestelldaten sortiert nach Packstückvolumen und Grundfläche der Orientierungen sowie die Containerdaten
        self.initial_modified_order_data, self.container_data = data_preparation(self.order_file_name, self.container_file_name)

        # Initialisiere die durch die Basisheuristik mit der initialen Packreihenfolge berechnete Lösung
        self.best_solution_base = Solution(copy.deepcopy(self.initial_modified_order_data), self.container_data) # erzeugte einer leere Lösung und Initailisiere sie mit der initialen MPS Packreihenfolge & den Containerdaten
        self.best_solution_base.calculate_solution() # Berechne die Lösung
        

        
        # Initialisiere Variable zum speichern der durch die Verbesserungsheuristik erzeugten besten Lösung
        self.best_solution_improve = 0

        # Initialisiere Listen zum speichern der Simulationsdaten durch die Grid_Search
        self.best_volume_usage_values = [] # Initialisiere eine Liste, welche die Volumenauslastung der besten Lösung jeder Generation speichert
        self.index_best_solution = [] # Initialisiere eine Liste, welche den Index des besten Individuums einer Population trägt

    
    # Berechne die benötigte Zeit
    def set_needed_time(self):
        self.end_time = time.time() # Schlusszeit
        self.time_needed = self.end_time - self.start_time # Berechne die benötigte Zeit


    # Funktion, die Eigenschaften über die übergebene Lösung ausgibt
    def print_solution_propertiers(self, best_solution):
        print("Die gesamten Versandkosten liegen bei " +  str(best_solution.costs) + " €")
        print("Die durchschnittliche Containerauslastung liegt bei " + str(best_solution.usage_average) + " %")
        print("Es wurden " + str(best_solution.container_number) + " Container benötigt")


    # Funktion, die den gesamten Algorithmus ausführt
    def run_algorithm(self):
            
        # Gebe die Ergebnisse der initialen Lösung aus
        print("Ergebnis der Basisheuristik mit der sortierten Packreihenfolge nach Volumen und größter Grundfläche")
        self.print_solution_propertiers(self.best_solution_base)

        # Visualisere gepackte Container der Lösung der Basisheuristik
        # self.best_solution_base.visualize()
        # Gebe Packplan der Lösung der Basisheuristik aus
        print("Packplan Basisheuristik")
        print(self.best_solution_base.containers)

        # Erzeuge eine Instanz des genetischen Algorithmus
        genetic_algorithm = Genetic_Algorithm(self.initial_modified_order_data, self.container_data, self.population_size, self.nr_of_iterations, self.choosen_parent_pairing, self.list_of_choosen_crossovers, self.nr_of_parents_to_keep)
        # Führe den genetischen Algorithmus aus
        self.best_solution_improve, self.best_volume_usage_values, self.index_best_solution = genetic_algorithm.run_genetic_algorithm()

        # Berechne die benötigte Zeit & gebe sie aus
        self.set_needed_time()
        print("Die Rechenzeit liegt bei " + str(round(self.time_needed, 2)) + " Sekunden\n")



        # Gebe die Ergebnisse der finalen Lösung mit der niedrigesten Volumenauslastung aus
        print("Ergebnisse der finalen Lösung:")
        self.print_solution_propertiers(self.best_solution_improve)

        print("\n Relative Verbesserung zum Ergebnis der Basisheuristik:")
        print(str((self.best_solution_improve.usage_average/self.best_solution_base.usage_average - 1) * 100) + "%\n")
        
        # Visualisere gepackte Container der besten Lösung
        self.best_solution_improve.print_packplan()
        self.best_solution_improve.visualize()
        # Gebe Packplan der besten Lösung aus
        print("Packplan der besten Lösung")
        print(self.best_solution_improve.containers)
        
        
        

        return self.best_volume_usage_values, self.index_best_solution, self.time_needed, self.best_solution_improve


### Class: Grid Search

In [114]:
#import multiprocessing as mp

# Klasse, in der verschiedene Ausprägungen der Parameter für den evolutionären Algorithmus definiert werden können. 
class Grid_Search:
    
    def __init__(self):
        # Definiere die zu simulierenden Benchmarkinstanzen
        self.list_of_order_instances = ["01_Datensatz_Beispiel_4"]
        # Definiere die Anzahl der Iteration
        self.grid_nr_of_iterations = 10
        # Definiere die Populationsgrößen
        self.list_of_population_sizes = [10]
        # Initialisiere Variable zur Auswahl der Verfahrens für die Bildung von Elternpaaren für den Crossover (0-5)
        self.list_of_choosen_parent_pairing = [0]
        # Initialisiere Liste zu Auswahl der durchzuführenden Crossovervarianten & befülle sie mit den folgenden Zahlen, um festzulegen mit welchen Crossovervarianten Nachkommen erzeugt werden sollen
        # 0 = erste bzw. zweite Hälfte der Packreihenfolge beider Elternteile zuerst
        # 1 = Container mit Auslastung > dem Auslastungsdurchschnitt zuerst
        # 2 = erste Hälfte der Packreihenfolge zuerst. Von A bzw. B zuerst
        self.list_of_lists_of_choosen_crossovers = [[1]]
        
        # Initialisiere Listen in welche die Ergebnisse aller Simulationen gespeichert werden
        self.list_volume_usage_values = []
        self. list_index_best_solution = []
        self.list_packplan = []

        # Initialisiere die aktuelle Zeit für den Dateinamen
        self.timestamp = datetime.now().strftime("%Y-%m-%d %Hh%Mm%Ss")

    # Ruft den gesamten Algorithmus für jede in der Gittersuche definierte Parameterkombination auf.
    def run_grid_search(self):

        # Führe Simulationen für die verschiedenen Parameterausprägungen durch
        for grid_order_instance in self.list_of_order_instances:
            for grid_population_size in self.list_of_population_sizes:
                for grid_list_of_choosen_crossovers in self.list_of_lists_of_choosen_crossovers:
                    for grid_choosen_parent_pairing in self.list_of_choosen_parent_pairing:
                        
                        # Definiere die Anzahl an Eltern, die für die Bildung der neuen Population berücksichtigt werden sollen
                        nr_of_parents_to_keep = 1 # Berücksichtige die beste Lösung der Vorgängerpopulation
                        # nr_of_parents_to_keep = round(grid_population_size/2) #/2 = Berücksichtige Hälfte der Vorgängerpopulationslösunegn

                        packing_algorithm = Packing_Algorithm(grid_order_instance, grid_population_size, self.grid_nr_of_iterations, grid_choosen_parent_pairing, grid_list_of_choosen_crossovers, nr_of_parents_to_keep) # Erzeuge eine Insatnz des Algorithmus mit den jeweiligen Parametern
                        best_volume_usage_values, index_best_solution, time_needed, best_solution_improve = packing_algorithm.run_algorithm() # Führe den Algorithmus aus und erhalte eine Liste mit der Volumenauslastung bzw. Index der besten Lösung je Population
                        properties_simulation = 'Time needed: ' + str(round(time_needed,2)) + ' - PopSize: ' + str(grid_population_size) + ' - Pairing: ' + str(grid_choosen_parent_pairing) + ' - Crossover: ' + ' '.join(str(element) for element in grid_list_of_choosen_crossovers) + ' - BI: ' + grid_order_instance #Speichere Eigenschaften der Simulation in einer Variable

                        # Füge die Eigenschaften der Simulation als erstes Elemnet den Listen mit den Simulationsergebnissen hinzu
                        best_volume_usage_values.insert(0, properties_simulation) 
                        index_best_solution.insert(0, properties_simulation)

                        # Füge die Listen mit den Simulationsergebnissen den Listen mit den Ergebnissen aller Simulationen hinzu
                        self.list_volume_usage_values.append(best_volume_usage_values)
                        self.list_index_best_solution.append(index_best_solution)
                        for i in range(len(packplan)):
                            self.list_packplan.append((packplan[i][0], packplan[i][1], packplan[i][2], packplan[i][3], packplan[i][4], packplan[i][5], packplan[i][6], packplan[i][7], packplan[i][8], packplan[i][9]))

        
            # Speichere die Ergebnisse der Simulation in eine CSV Datei
            #self.save_to_csv(grid_order_instance) 

    # Funktion, die alle übergebenen Ergebnisse einer Simulation in eine CSV Datei speichert
    def save_to_csv(self, grid_order_instance):
       
        # Initialisiere die Dateipfade in  welche die Ergebnisse gespeichert werden sollen
        file_path_volume = 'Simulationen//' + str(self.timestamp) + ' BI' + grid_order_instance[:2] + ' Simulation Volumenauslastung mit Mutation.csv'
        file_path_index = 'Simulationen//' + str(self.timestamp) + ' BI' + grid_order_instance[:2] + ' Simulation bester Index mit Mutation.csv'
        file_path_packplan = 'Simulationen//' + str(self.timestamp) + ' BI' + grid_order_instance[:2] + ' Simulation Packplan.csv'

        # Erzeuge Dateframes mit den Simulationsergebnissen
        results_volume_usage = pd.DataFrame(self.list_volume_usage_values)
        self.list_volume_usage_values = []
        results_volume_usage = results_volume_usage.T
        result_index_best_solutions = pd.DataFrame(self.list_index_best_solution)
        self.list_index_best_solution = []
        result_index_best_solutions = result_index_best_solutions.T
        results_packplan = pd.DataFrame(self.list_packplan)
        self.list_packplan = []

        # Speichere die Dataframes in CSV Dateien
        # results_volume_usage.to_csv(path_or_buf=file_path_volume, header=False, index=False)
        # result_index_best_solutions.to_csv(path_or_buf=file_path_index, header=False, index=False)
        results_packplan.to_csv(path_or_buf=file_path_packplan, header=False, index=False)
    


## 1. Datenvorverarbeitung

In [115]:
# Datenvorverarbeitung
def data_preparation (order_file_name, container_file_name):
    
    '''Importiere  Bestelldaten und Containerdaten'''
    
    # Lese Containerdaten ein
    container_list = pd.read_excel(container_file_name, sheet_name = "Container", skiprows = 1)
    
    # Erzeuge eine Liste für die Containertypen und speichere die relevanten Informationen darin ab
    container_data = []
    for i in range(len(container_list)):
        container_type = (container_list["Länge innen [mm]"][i],container_list["Breite innen [mm]"][i],container_list["Höhe innen [mm]"][i], 
            round(container_list["Fixkosten gesamt"][i],2), round(container_list["Max.Gewicht [kg]"][i],2))
        container_data.append(container_type)

    # Lese die Bestelldaten ein
    order_list = pd.read_excel(order_file_name)
    
    # Erzeuge eine Liste für die Packstücke der Bestellung und speichere die relevanten Informationen darin ab
    start = 0 # Hilfsvariable für Tabellenindex
    end = start + 1 # Hilfsvariable für Tabellenindex
    order_data = [] 
    for i in range(0, len(order_list)):
        packaging_unit = (int(order_list[start:end]["Länge [mm]"]),int(order_list[start:end]["Breite [mm]"]), int(order_list[start:end]["Höhe [mm]"]), float(order_list[start:end]["Gewicht [kg]"]), 
                          str(order_list["Orientierungen"][start]))
        order_data.append(packaging_unit)
        start = start + 1
        end = end + 1
    '''Bestelldaten und Containerdaten importiert'''
    
    '''Erzeuge die modifizierte Bestelldaten'''
    
    modified_order_data = [] # Liste zur Speicherung der modifizierten Bestelldaten
    ID = 0 # Fortlaufende Variable zur Zuweisung der Packstück ID
    
    # Durchlaufe die Packstücke aus den Bestelldaten
    for i in range(len(order_data)):
        
            # Rufe die Packstückdimensionen ab
            unit_dimensions = (order_data[i][0],order_data[i][1],order_data[i][2]) 
            # Sortiere die Packstückdimensionen absteigend
            sorted_unit_dimensions = sorted(unit_dimensions, reverse = True)
            # Erzeuge sämtliche Orientierungsmöglichkeiten
            all_orientations = list(itertools.permutations(sorted_unit_dimensions)) 
            
            # Berücksichtige nun die Orientierungsrestriktionen und entferne verbotene Orientierungen            
            # Rufe zunächst die zugelassenen Orientierungen ab
            orientations_permitted = list(order_data[i][4]) 
            # Erzeuge eine Liste, in der die Indizes der verbotenen Orientierungen abgespeichert werden
            index_to_drop = [] 
            for j in range(1, 7): # Suche nach Kennziffern 1 - 6 
                if str(j) not in orientations_permitted: # Wenn Kennziffer nicht erlaubt, dann füge den Index der Liste hinzu
                    index_to_drop.append((j-1)) #Kennziffer 1 steht an Position 0. Daher j - 1.
            # Sortiere jetzt die Indizes die zu entfernen sind
            index_to_drop = sorted(index_to_drop, reverse = True)
            # Entferne  nun die verbotenen Orientierungen
            for itd in index_to_drop:
                all_orientations.pop(itd) 
            
            # Erzeuge ein modifiziertes Packstücke MPS
            MPS = [] 
            
            ID = i + 1 # Gebe Packstücken eine eindeutige ID. Dies ermöglicht eine Unterscheidung von identischen Packstücken.
            for orientation in all_orientations:
                area = (orientation[0] * orientation[1]) / 100 # Errechne die Grundfläche einer Orientierung
                volume = (orientation[0] * orientation[1] * orientation [2]) / 1000 # Errechne das Packstückvolumen
                # Füge dem MPS für jede zugelassene Orientierung folgende Informationen hinzu: 
                # Grundfläche, Packstück ID, Länge, Breite, Höhe, Gewicht, Volumen
                MPS_position = (area, ID, orientation[0], orientation[1], orientation[2], order_data[i][3], round(volume, 2))
                MPS.append(MPS_position)

            # Füge das MPS den modfizierten Bestelldaten hinzu
            modified_order_data.append(MPS)
            
    # Sortiere jedes MPS absteigend nach der Grundfläche der Orientierungsvarianten
    for i, MPS in enumerate(modified_order_data):
        modified_order_data[i] = sorted(MPS, key=lambda elem: elem[0], reverse = True) 
    
 
    # Sortiere die MPS nach Volumen
    sorted_modified_order_data = [] # Liste in der die sortierten MPS gespeichert werden
    volume_index_old = [] # Volumen und aktuellen Index aller MPS der unsortierten modifizierten Bestelldaten abspeichern
    index = 0
    for MPS in modified_order_data:
        volume_index = (MPS[0][6], index)
        volume_index_old.append(volume_index)
        index += 1
    
    # Sortiere die erzeugte Liste
    volume_index_new = sorted (volume_index_old, reverse = True) 
    
    # Nun ist eine Zuordnung zwischen den Indizes der unsortierten Bestelldaten und den nach Volumen sortierten modifzierten Bestelldaten möglich
    for index_to_change in volume_index_new:
        sorted_modified_order_data.append(modified_order_data[index_to_change[1]]) # Index steht an Position 1
        
    '''Modifizierte Bestelldaten erzeugt'''

    # Gebe Liste mit Instanze aus modifizierte Bestelldaten mit zufälliger Reihenfolge und Containerdaten zurück
    return sorted_modified_order_data, container_data 

## 2. Restriktionen in Cython

### 2.1 Platzierungsbedingung

In [116]:
%%cython

def Check_Placement_Modified_Stacking ((int, int, int) container, float max_weight, float carried_weight, article_locations, (float, int, int, int, int, float, float) orientation, int stack_raw_width, int stack_length, unsigned int stack_box_length, int stack_box_height, int stack_box_width):

    center_of_gravity_width = orientation[2]/2 # Schwerpunktbreite 
    center_of_gravity_length = orientation[3]/2 # Schwerpunktlänge
    cdef int add_allowed = 0

    # Prüfe ob das Packstück auf den Stapel passt
    if ((center_of_gravity_width <= stack_raw_width) and # Schwerpunkt darf nicht die Stapelbreite überschreiten
        (center_of_gravity_length <= stack_length) and # Schwerpunkt darf nicht die Stapellänge überschreiten
        (orientation[4] + stack_box_height <= container[1]) and # Containerhöhe darf nicht überschritten werden
        (orientation[3] + stack_box_length <= container[2]) and # Containerlänge darf nicht überschritten werden
        (orientation[2] + stack_box_width <= container[0]) and # Containerbreite darf nicht überschritten werden
        (carried_weight + orientation[5] <= max_weight)): # Gewichtsobergrenzte darf nicht überschritten werden

        add_allowed = True

        # Prüfe, ob Überschneidungen zu anderen Packstücke existieren
        for article in article_locations:
            if ((((article.y <= stack_box_height) and (article.height + article.y > stack_box_height)) or ((article.y > stack_box_height) and (article.y < stack_box_height + orientation[4])))and # Überschneidungen in der Höhe
                (((article.x > stack_box_width) and (article.x < stack_box_width + orientation[2])) or ((article.x <= stack_box_width) and (article.x + article.width > stack_box_width))) and # Überschneidungen in der Breite
                (((article.z > stack_box_length) and (article.z < stack_box_length + orientation[3])) or ((article.z <= stack_box_length) and (article.z +article.length > stack_box_length)))): # Überschneidungen in der Länge
                    
                add_allowed = False
                break

    return add_allowed

In [117]:
%%cython

def Check_Placement (article_to_add, articles_in_raw, (int, int, int) container, float max_weight, float carried_weight, (float, int, int, int, int, float, float) orientation, unsigned int raw_width, int raw_length, unsigned int box_length, unsigned int box_height, float factor_overfilling):

    # cdef int add_allowed = 0
    # add_allowed: cython.int = 0
    cdef int add_allowed = 0


    if ((orientation[2] <= raw_width) and # Breite der Orientierung muss kleiner gleich der Breite des Leerraums sein
        (orientation[3] <= raw_length * factor_overfilling) and # Länge der Orientierung darf Reihenlänge bis zu einer gewissen Toleranz T nicht überschreiten. T = factor_overfilling.
        (orientation[4] + box_height <= container[1]) and # Höhe der Orientierung zzgl. der Höhenposition muss kleiner gleich der Containerhöhe sein
        (orientation[3] + box_length <= container[2]) and # Länge der Orientierung zzgl. der Längenpoistion darf Containerlänge nicht überschreiten
        ((carried_weight + orientation[5]) <= max_weight)): # Sicherstellung der Gewichtseinhaltung

            # Stelle sicher, dass zu betrachtetes Packstück noch nicht gepackt wurde
            # Bedingung wird aufgrund der erzeugten Nebenreihen benötigt
            add_allowed = article_to_add not in articles_in_raw

    return add_allowed

### 2.2 Bedingung für Traglast einzelner Packstücke

In [118]:
%%cython

def Calculate_Overlapping_Areas(selected_orientation, articles_from_extension):
    # Liefert für jedes Packstück der Stapelgrundfläche, auf das gestapelt wird, zu welchem Anteil es von dem zu packenden Packstück überdeckt wird
    
    # Definiere Datentypen
    cdef int width_article_to_pack
    cdef int length_article_to_pack
    cdef float width_coordinate_article_to_pack
    cdef float length_coordinate_article_to_pack
    
    cdef int package_ID_article_base
    cdef int width_article_base
    cdef int length_article_base
    cdef float width_coordinate_article_base
    cdef float length_coordinate_article_base

    cdef float overlapping_area
    cdef float overlap_in_weight
    cdef float overlap_in_length

    # Initialisiere Listen zum Speichern der Packstück ID aller Packstücke, welche durch das zu packende Packstück überdeckt werden + die Göße der übderdeckten Fläche
    overlapping_area_of_each_base_article  = []

    
    # Initialisiere Variablen für die Maße und Koordinaten des zu packenden Packstücks
    width_article_to_pack = selected_orientation[2] #0
    length_article_to_pack = selected_orientation[0] #2
    width_coordinate_article_to_pack = selected_orientation[3] #3
    length_coordinate_article_to_pack = selected_orientation[5]

    for article in articles_from_extension: # Durchlaufe alle Packstück der Stapelgrundfläche
        
        # Initialisiere Variablen für die Maße, Koordinaten und ID des zu Packstücks aus der Fläche, die als Basis für den Stapel dient
        package_ID_article_base = article[0] # Packstück ID
        width_article_base = article[1] # Packstück Breite
        length_article_base = article[2] # Packstück Länge
        width_coordinate_article_base = article[3] # Packstück Breitenkoordinate
        length_coordinate_article_base = article[4] # Packstück Längenkoordinate

        # Initialisiere Variable für die Überlappende Fläche
        overlapping_area = 0
        
    
        # Brechne die Überlappung in der Breite
        overlap_in_weight = min((width_coordinate_article_base + width_article_base), (width_coordinate_article_to_pack + width_article_to_pack)) - max(width_coordinate_article_base, width_coordinate_article_to_pack)
        # Berechne die Überlappung in der Länge
        overlap_in_length = min((length_coordinate_article_base + length_article_base), (length_coordinate_article_to_pack + length_article_to_pack)) - max(length_coordinate_article_base, length_coordinate_article_to_pack)
        
        if (overlap_in_weight >= 0) and (overlap_in_length >= 0):
            overlapping_area = overlap_in_weight * overlap_in_length # Berechnen die überlappende Fläche
        
        if (overlapping_area > 0): # Überprüfe, ob es eine überlappende Fläche gibt
            # Füge Informationen über das Packstück der Stapelgrundfläche der Liste hinzu
            overlapping_area_of_each_base_article.append((package_ID_article_base, overlapping_area))
   


    return overlapping_area_of_each_base_article

## 3. Anwednung und Ergebnisausgabe

In [119]:
# # Ausführen des Algorithmus mit Line-by-Line-Profiler 
# %load_ext line_profiler

# # Erzeuge eine Instanz des Algorithmus mit den gewünschten Paramtern (Dateiname der Bestelldaten, Populationsgröße, Variante der Paarungsselektion, Variante des Crossover, Anzahl zu übernehmende beste Individuen in Folgepopulation)
# packing_algorithm = Packing_Algorithm("06_Datensatz_50-75 Artikel_leicht_heterogen2", 10, 10, 0, [0], 1)
# # Ruf Algorithmus mit Profiler auf und definiere für welche Funktion die Zeit gemessen werden soll
# %lprun -f Container.run_Wall_Building packing_algorithm.run_algorithm()

In [120]:
# Starte Simulation mit verschiedenen Parametern
simulation = Grid_Search() # Erzeuge eine Instanz der Gittersuche
simulation.run_grid_search() # Führe Simulationen mit den in der Gittersuche definierten Parametern durch

PermissionError: [Errno 13] Permission denied: '01_Datensatz_Beispiel_4.xlsx'